# Matter-Antimatter asymmetries at the LHC
## Lab Experiment

Author: Ahmad Abdelhakam Mahmoud

### Configuration
The cell below imports all required Python modules.

In [1]:
import uproot
import numpy as np
import matplotlib
import matplotlib.pylab as plt
import matplotlib.colors as colors
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from matplotlib.colors import LogNorm
from matplotlib.colors import Normalize
from matplotlib import pyplot as mp

from scipy.optimize import curve_fit # https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html
plt.rcParams.update({
    "font.family": "serif",
    'font.size':10,
    'lines.linewidth':0.5,
    'axes.spines.right':False,
    'axes.spines.top':False,
    'lines.markersize': 2
})

import scipy.constants as constants

h_bar = 197.326 #MeVfm

### Input data
This opens input data files and assigns them to variables.

In [2]:
path = '/shared/' # set this to '' to run on the GitHub version
events_sim = uproot.open(path+'PhaseSpaceSimulation.root')
events_down = uproot.open(path+'B2HHH_MagnetDown.root')
events_up = uproot.open(path+'B2HHH_MagnetUp.root')

In [3]:
#rest masses in MeV/c²
pion_rest_mass_mev = 139.5
kaon_rest_mass = 493.677

### Selecting data and calculating derived quantities
The following cell reads the input data. This is where all derived quantities (such as transverse momentum) are calculated and where selection criteria are applied (such as the z component of the momentum being positive).

In [ ]:
# Check what's in the tree. 
# Note that the simulation tree is called 'PhaseSpaceTree' and does not have the ProbPi/K variables filled.
print('Input data variables:')
print(events_up['DecayTree'].keys())

# These are the arrays to hold the data
pT = []
pX_1 = []
pX_2 = []
pX_3 = []
pY_1 = []
pY_2 = []
pY_3 = []
pZ_1 = []
pZ_2 = []
pZ_3 = []
ProbK_1 = []
ProbPi_1 = []
ProbK_2 = []
ProbPi_2 = []
ProbK_3 = []
ProbPi_3 = []
Charge_1 = []
Charge_2 = []
Charge_3 = []
# A counter for bookkeeping
event_counter = 0

# If set to a value greater than 0, limits the number of events analysed
# Set to -1 to run over all events. 
# It is recommended to keep the number of events limited while developing the analysis.


# Select which set of input data is to be analysed. Uncomment exactly one line
#trees = [events_sim['PhaseSpaceTree']]                       # Simulation
trees_down = [events_down[b'DecayTree']]                          # Magnet down data
trees_up = [events_up['DecayTree']]                             # Magnet up data
trees = [events_down[b'DecayTree'],events_up['DecayTree']]   # Magnet down+up datac

# This loop goes over the trees to be analysed
for tree in trees:
    # This outer loop is a technical loop of uproot over chunks of events
    for data in tree.iterate([b'H*_P[XYZ]',b'H*_Charge',b'H*_Prob*',b'H*_isMuon']):
        # As Python can handle calculations with arrays, we can calculate derived quantities here
        pT_H1 = np.sqrt(data[b'H1_PX']**2+data[b'H1_PY']**2)
        pT_H2 = np.sqrt(data[b'H2_PX']**2+data[b'H2_PY']**2)
        pT_H3 = np.sqrt(data[b'H3_PX']**2+data[b'H3_PY']**2)
        
        # This loop will go over individual events
        for i in range(0,len(data[b'H1_PZ'])):
            event_counter += 1
            # Decide here which events to analyse
            if (data[b'H1_PZ'][i] < 0) or (data[b'H2_PZ'][i] < 0) or (data[b'H3_PZ'][i] < 0): continue
            if (data[b'H1_isMuon'][i] > 0) or (data[b'H2_isMuon'][i] > 0) or (data[b'H3_isMuon'][i] > 0): continue

            # Fill arrays of events to be plotted and analysed further below
            # Adding values for all three hadrons to the same variable here
            pT.append(pT_H1[i])
            pT.append(pT_H2[i])
            pT.append(pT_H3[i])
            pX_1.append(data[b'H1_PX'][i])
            pX_2.append(data[b'H2_PX'][i])
            pX_3.append(data[b'H3_PX'][i])
            pY_1.append(data[b'H1_PY'][i])
            pY_2.append(data[b'H2_PY'][i])
            pY_3.append(data[b'H3_PY'][i])
            pZ_1.append(data[b'H1_PZ'][i])
            pZ_2.append(data[b'H2_PZ'][i])
            pZ_3.append(data[b'H3_PZ'][i])
            
            ProbK_1.append(data[b'H1_ProbK'][i])
            ProbPi_1.append(data[b'H1_ProbPi'][i])
            ProbK_2.append(data[b'H2_ProbK'][i])
            ProbPi_2.append(data[b'H2_ProbPi'][i])
            ProbK_3.append(data[b'H3_ProbK'][i])
            ProbPi_3.append(data[b'H3_ProbPi'][i])
            
            Charge_1.append(data[b'H1_Charge'][i])
            Charge_2.append(data[b'H2_Charge'][i])
            Charge_3.append(data[b'H3_Charge'][i])
 

            
print('Read {:d} events'.format(event_counter))

Input data variables:
[b'B_FlightDistance', b'B_VertexChi2', b'H1_PX', b'H1_PY', b'H1_PZ', b'H1_ProbK', b'H1_ProbPi', b'H1_Charge', b'H1_IPChi2', b'H1_isMuon', b'H2_PX', b'H2_PY', b'H2_PZ', b'H2_ProbK', b'H2_ProbPi', b'H2_Charge', b'H2_IPChi2', b'H2_isMuon', b'H3_PX', b'H3_PY', b'H3_PZ', b'H3_ProbK', b'H3_ProbPi', b'H3_Charge', b'H3_IPChi2', b'H3_isMuon']


In [ ]:
kaon_rest_mass_mev = 493.677
kaon_rest_mass_mev_unc = 0.016
pion_rest_mass_mev = 139.57039
pion_rest_mass_mev_unc = 0.00018

### Data Visualization
The cell below produces histograms of the data contained in the arrays that were filled in the cell above.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))
fig.subplots_adjust(wspace=0.3) # increase horizontal space between plots

ProbK = []
ProbPi = []
ProbK = np.append(ProbK,ProbK_1)
ProbK = np.append(ProbK,ProbK_2)
ProbK = np.append(ProbK,ProbK_3)
ProbPi = np.append(ProbPi,ProbPi_1)
ProbPi = np.append(ProbPi,ProbPi_2)
ProbPi = np.append(ProbPi,ProbPi_3)


values_pi,bins_pi,patches_pi = ax[0].hist(ProbK, bins = 200, range = [0, 1],histtype='step',label='K meson',color = 'k')
ax[0].hist(ProbPi, bins = 200, range = [0, 1],histtype='stepfilled',alpha=0.3,label='$\pi$ Meson',color = 'r')
ax[0].set_xlabel(r'Probability of mesons')
ax[0].set_ylabel('Entries per 100MeV')
ax[0].grid(True,ls=':')
ax[0].set_title('Kaon-Pion Probability Distributions')
ax[0].legend()

# This plots a 2D-histogram with values converted to GeV and with a logarithmic colour scale

Pi2K = ax[1].hist2d(ProbPi, ProbK, bins = [100,100], range = [[0,1],[0,1]],norm=colors.LogNorm())
ax[1].set_xlabel(r'probability $\pi$ Mesons')
ax[1].grid(True,ls=':')
ax[1].set_ylabel(r'probability K Mesons')
ax[1].set_title('Kaon-Pion Probability Histogram')
fig.colorbar(Pi2K[3],ax=ax[1]) # let's add the colour scale
plt.savefig('Pi2K',dpi = 500)
plt.show()

In [ ]:
plt.figure(figsize=(3, 3), dpi=300)
# This plots a 2D-histogram with values converted to GeV and with a logarithmic colour scale
Pi2K = plt.hist2d(ProbPi, ProbK, bins = [100,100], range = [[0,1],[0,1]],norm=colors.LogNorm())
plt.colorbar(Pi2K[3]) # let's add the colour scale
plt.xlabel(r'probability $\pi$ Mesons')
plt.grid(True,ls=':')
plt.ylabel(r'probability K Mesons')
plt.show()

 ### Invariant Mass Reconstruction

In [ ]:
%%capture
pX_1 = np.array(pX_1)
pX_2 = np.array(pX_2)
pX_3 = np.array(pX_3)

pY_1 = np.array(pY_1)
pY_2 = np.array(pY_2)
pY_3 = np.array(pY_3)

pZ_1 = np.array(pZ_1)
pZ_2 = np.array(pZ_2)
pZ_3 = np.array(pZ_3)

ProbK_1 = np.array(ProbK_1)
ProbPi_1 = np.array(ProbPi_1)

ProbK_2 = np.array(ProbK_2)
ProbPi_2 = np.array(ProbPi_2)

ProbK_3 = np.array(ProbK_3)
ProbPi_3 = np.array(ProbPi_3)

Charge_1 = np.array(Charge_1)
Charge_2 = np.array(Charge_2)
Charge_3 = np.array(Charge_3)

kaon_all_probs = [ProbK_1,ProbK_2,ProbK_3]
pion_all_probs = [ProbPi_1,ProbPi_2,ProbPi_3]
data = np.vstack([pion_all_probs,kaon_all_probs])

momentum_mag_H1 = np.sqrt(np.square(pX_1) + np.square(pY_1) + np.square(pZ_1))
momentum_mag_H2 = np.sqrt(np.square(pX_2) + np.square(pY_2) + np.square(pZ_2))
momentum_mag_H3 = np.sqrt(np.square(pX_3) + np.square(pY_3) + np.square(pZ_3))


def find_individual_energies (momentum, rest_mass):
    
    return np.sqrt(np.square(rest_mass) + np.square(momentum))

def find_individual_energies_unc (p, m, p_unc, m_unc):
    
    return np.sqrt(np.square(p_unc*p*(1/np.sqrt(np.square(m) + np.square(p)))+np.square(m_unc*m*(1/np.sqrt(np.square(m) + np.square(p))))))

energies_kaon_H1 = find_individual_energies(momentum_mag_H1, kaon_rest_mass_mev)
energies_kaon_H2 = find_individual_energies(momentum_mag_H2, kaon_rest_mass_mev)
energies_kaon_H3 = find_individual_energies(momentum_mag_H3, kaon_rest_mass_mev)

def find_invariant_mass():
    
    invariant_mass = (np.sqrt(
        np.square(energies_kaon_H1+energies_kaon_H2+energies_kaon_H3) - 
        (np.square((pX_1)+(pX_2)+(pX_3)) + np.square((pY_1)+(pY_2)+(pY_3)) + 
         np.square((pZ_1)+(pZ_2)+(pZ_3)))))
    
    return invariant_mass

invariant_mass_B_meson = find_invariant_mass()

In [ ]:
# This is using the simulation
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 6))
#fig.subplots_adjust(wspace=0.3) # increase horizontal space between plots

# This plots two 1D-histograms.pX_1
# The color is changed automatically, the styles are set by hand
# keep hold of the pT histogram data for fitting later
ax.hist(invariant_mass_B_meson, bins = 200, range = [5270,5285],histtype='step',label='$Entries$')
ax.set_xlabel('Invariant Mass of The B Meson')
ax.set_ylabel('Entries per 100 MeV')
ax.legend()
plt.savefig('pTpZ.pdf')

## Particle Selection

### Filtering Conditions

These are the conditions we use to filter the data. Lambda functions are used, taking in `ProbK,ProbPi`, then applying filters as will be visualised in a later section. Lambda functions are used to simplify the process later.

In [ ]:
kaon_condition = lambda ProbK, ProbPi: (ProbK >= 0.8) & (ProbPi <= 0.2) | ((ProbK >= 0.8) & (ProbPi <= 0.3) & (ProbPi >= 0.2))
pion_condition = lambda ProbK, ProbPi: (ProbPi >= 0.59) & (ProbK <= 0.35)


In [ ]:
#rest masses in MeV/c²
pion_rest_mass_mev = 139.57
kaon_rest_mass = 493.677


Here is the code where the filtering happens. We create filtering conditions for each track ```H1,H2,H3``` individually, for pions and kaons. Using the vectorisation ability of numpy arrays, 2 intermediary arrays are created that dstore every event where a kaon is- called ```kaons```- and for pions ```pions```. Using these intermediary arrays and ```np.where```, a final mask is made that contains all the indices where a kaon and 2 pions are found. This array, called ```indices``` are all the events we are interested in.

In [ ]:
# Apply conditions to each hadron
kaon_1 = kaon_condition(ProbK_1, ProbPi_1)
pion_1 = pion_condition(ProbK_1, ProbPi_1)

kaon_2 = kaon_condition(ProbK_2, ProbPi_2)
pion_2 = pion_condition(ProbK_2, ProbPi_2)

kaon_3 = kaon_condition(ProbK_3, ProbPi_3)
pion_3 = pion_condition(ProbK_3, ProbPi_3)

# Count kaons and pions in each event
kaons = kaon_1.astype(int) + kaon_2.astype(int) + kaon_3.astype(int)
pions = pion_1.astype(int) + pion_2.astype(int) + pion_3.astype(int)

# Find events where there is one kaon and two pions
indices = np.where((kaons == 1) & (pions == 2))[0]

print("Indices of events corresponding to the decay B+- -> K, pi, pi:", len(indices))



In [ ]:
prob_K_filter = np.array([])
prob_K_filter = np.append(prob_K_filter,ProbK_1[indices])
prob_K_filter = np.append(prob_K_filter,ProbK_2[indices])
prob_K_filter = np.append(prob_K_filter,ProbK_3[indices])

prob_Pi_filter = np.array([])
prob_Pi_filter = np.append(prob_Pi_filter,ProbPi_1[indices])
prob_Pi_filter = np.append(prob_Pi_filter,ProbPi_2[indices])
prob_Pi_filter = np.append(prob_Pi_filter,ProbPi_3[indices])

In [ ]:
plt.figure(figsize=(3, 3), dpi=300)
# This plots a 2D-histogram with values converted to GeV and with a logarithmic colour scale
Pi2K = plt.hist2d(prob_Pi_filter, prob_K_filter, bins = [100,100], range = [[0,1],[0,1]],norm=colors.LogNorm())
plt.colorbar(Pi2K[3]) # let's add the colour scale
plt.xlabel(r'probability $\pi$ Mesons')
plt.grid(True,ls=':')
plt.ylabel(r'probability K Mesons')
plt.show()

In the below block, the kaon and 2 pion arrays are made.
1. First, we create boolean arrays from the kaon$_i$ arrays $(i \in {1,2,3})$. These multi-dimensonal arrays are of the size $\begin{pmatrix}3\\N\end{pmatrix}$. Empty lists are then created for the particles of interest.

2. The loop `for i in idx:` iterates over all the events of interest (where one kaon and two pions are found). Then, ```np.where()``` is used to see which hadrons in the event number =```idx``` satisfy the conditions. The ```[0][0]``` indexing is used, as there is only be one kaon per event, so we regard only the kaon found. 

3. The same method is done in ```pion_idx``` in order to find the two remaining pions. However, we do not need to index ```[0][0]``` as we are interested in both events the ```np.where``` function returns.

4. The kaon and pion momenta components for all three particles are stored in the ```{hadron}_momentum``` array, which is then appended to the ```{hadron}_momenta``` array. This can then be sorted in order to find the components.

In [ ]:
# Gather the kaon and pion conditions into arrays
kaon_conditions = np.array([kaon_1, kaon_2, kaon_3])
pion_conditions = np.array([pion_1, pion_2, pion_3])

# Initialize lists to store momenta
kaon_momenta = []
pion_momenta_1 = []
pion_momenta_2 = []

# Loop over the indices of events
for idx in indices:
    # Find the kaon and pions in this event
    kaon_idx = np.where(kaon_conditions[:, idx])[0][0]
    pion_idxs = np.where(pion_conditions[:, idx])[0]
    
    # Gather the momentum for the kaon
    kaon_momentum = np.array([globals()[f'pX_{kaon_idx+1}'][idx], globals()[f'pY_{kaon_idx+1}'][idx], globals()[f'pZ_{kaon_idx+1}'][idx]])
    kaon_momenta.append(kaon_momentum)
    
    # Gather the momenta for the pions
    pion_momentum_1 = np.array([globals()[f'pX_{pion_idxs[0]+1}'][idx], globals()[f'pY_{pion_idxs[0]+1}'][idx], globals()[f'pZ_{pion_idxs[0]+1}'][idx]])
    pion_momenta_1.append(pion_momentum_1)
    
    pion_momentum_2 = np.array([globals()[f'pX_{pion_idxs[1]+1}'][idx], globals()[f'pY_{pion_idxs[1]+1}'][idx], globals()[f'pZ_{pion_idxs[1]+1}'][idx]])
    pion_momenta_2.append(pion_momentum_2)


# Convert lists to numpy arrays
kaon_momenta = np.array(kaon_momenta)
pion_momenta_1 = np.array(pion_momenta_1)
pion_momenta_2 = np.array(pion_momenta_2)

In [ ]:
# Initialize lists to store pion plus and minus momenta
pion_plus_momenta = []
pion_minus_momenta = []

# Loop over the indices of the events to sort pions into plus and minus
for idx in range(len(pion_momenta_1)):
    if Charge_1[idx] == 1:
        pion_plus_momentum = pion_momenta_1[idx]
        pion_minus_momentum = pion_momenta_2[idx]
    elif Charge_1[idx] == -1:
        pion_plus_momentum = pion_momenta_2[idx]
        pion_minus_momentum = pion_momenta_1[idx]
    else:
        # If the first pion is not charged, then check the second
        if Charge_2[idx] == 1:
            pion_plus_momentum = pion_momenta_2[idx]
            pion_minus_momentum = pion_momenta_1[idx]
        elif Charge_2[idx] == -1:
            pion_plus_momentum = pion_momenta_1[idx]
            pion_minus_momentum = pion_momenta_2[idx]
    
    pion_plus_momenta.append(pion_plus_momentum)
    pion_minus_momenta.append(pion_minus_momentum)

# Convert the lists to numpy arrays
pion_plus_momenta = np.array(pion_plus_momenta)
pion_minus_momenta = np.array(pion_minus_momenta)



In [ ]:
# Initialize list to store charges
kaon_charges = []

# Loop over the indices of events
for idx in indices:
    # Find the kaon in this event
    kaon_idx = np.where(kaon_conditions[:, idx])[0][0]
    
    # Gather the charge for the kaon
    kaon_charge = globals()[f'Charge_{kaon_idx+1}'][idx]
    kaon_charges.append(kaon_charge)

# Convert list to numpy array
kaon_charges = np.array(kaon_charges)

Then the momenta can just be separated to their components.

Arbitrarily decide pion 1 is pion plus and pion 2 is pion minus

In [ ]:
# Kaon momentum components
kaon_momenta_x = kaon_momenta[:, 0]
kaon_momenta_y = kaon_momenta[:, 1]
kaon_momenta_z = kaon_momenta[:, 2]

kaon_momenta_unc_x = 0.005*kaon_momenta_x # momentum uncertainty of 0.5% introduced
kaon_momenta_unc_y = 0.005*kaon_momenta_y
kaon_momenta_unc_z = 0.005*kaon_momenta_z

# First pion momentum components
pion_momenta_1_x = pion_plus_momenta[:, 0]
pion_momenta_1_y = pion_plus_momenta[:, 1]
pion_momenta_1_z = pion_plus_momenta[:, 2]

pion_momenta_1_unc_x = 0.005*pion_momenta_1_x # momentum uncertainty of 0.5% introduced
pion_momenta_1_unc_y = 0.005*pion_momenta_1_y
pion_momenta_1_unc_z = 0.005*pion_momenta_1_z

# Second pion momentum components
pion_momenta_2_x = pion_minus_momenta[:, 0]
pion_momenta_2_y = pion_minus_momenta[:, 1]
pion_momenta_2_z = pion_minus_momenta[:, 2]

pion_momenta_2_unc_x = 0.005*pion_momenta_2_x
pion_momenta_2_unc_y = 0.005*pion_momenta_2_y
pion_momenta_2_unc_z = 0.005*pion_momenta_2_z


## The Three Body Invariant Mass

The below two code blocks find the invariant mass for the $B^{\pm}$ distribution. The full dsitribution is then plotted.

In [ ]:
def find_abs_momenta(px,py,pz):
    return np.sqrt(np.square(px)+np.square(py)+np.square(pz))

def find_abs_momenta_unc(px,py,pz,px_unc,py_unc,pz_unc):
    return np.sqrt(np.square((1/np.sqrt(np.square(px)+np.square(py)+np.square(pz)))*px*px_unc)+np.square((1/np.sqrt(np.square(px)+np.square(py)+np.square(pz)))*py*py_unc)+np.square((1/np.sqrt(np.square(px)+np.square(py)+np.square(pz)))*pz*pz_unc))

kaon_p_abs = find_abs_momenta(kaon_momenta_x,kaon_momenta_y,kaon_momenta_z)
kaon_p_abs_unc = find_abs_momenta_unc(kaon_momenta_x,kaon_momenta_y,kaon_momenta_z,kaon_momenta_unc_x,kaon_momenta_unc_y,kaon_momenta_unc_z)

pion_1_p_abs = find_abs_momenta(pion_momenta_1_x,pion_momenta_1_y,pion_momenta_1_z)
pion_1_p_abs_unc = find_abs_momenta_unc(pion_momenta_1_x,pion_momenta_1_y,pion_momenta_1_z,pion_momenta_1_unc_x,pion_momenta_1_unc_y,pion_momenta_1_unc_z)

pion_2_p_abs = find_abs_momenta(pion_momenta_2_x,pion_momenta_2_y,pion_momenta_2_z)
pion_2_p_abs_unc = find_abs_momenta_unc(pion_momenta_2_x,pion_momenta_2_y,pion_momenta_2_z,pion_momenta_2_unc_x,pion_momenta_2_unc_y,pion_momenta_2_unc_z)

In [ ]:
kaon_energies = find_individual_energies(kaon_p_abs, kaon_rest_mass_mev)
kaon_energies_unc = find_individual_energies_unc(kaon_p_abs, kaon_rest_mass_mev,kaon_p_abs_unc, kaon_rest_mass_mev_unc)

pion_1_energies = find_individual_energies(pion_1_p_abs, pion_rest_mass_mev)
pion_1_energies_unc = find_individual_energies_unc(pion_1_p_abs, pion_rest_mass_mev,pion_1_p_abs_unc, pion_rest_mass_mev_unc)

pion_2_energies= find_individual_energies(pion_2_p_abs, pion_rest_mass_mev)
pion_2_energies_unc = find_individual_energies_unc(pion_2_p_abs, pion_rest_mass_mev,pion_2_p_abs_unc, pion_rest_mass_mev_unc)

def find_invariant_mass_new(H1_energies,H2_energies,H3_energies,H1_momenta,H2_momenta,H3_momenta):
    
    invariant_masses = np.sqrt(np.square(H1_energies+H2_energies+H3_energies) - (
        np.square(H1_momenta)+np.square(H2_momenta)+np.square(H3_momenta)))
    
    return invariant_masses

def quadrature(p1_unc,p2_unc,p3_unc):    
    return np.sqrt(np.square(p1_unc)+np.square(p2_unc)+np.square(p3_unc))

px_unc = quadrature(kaon_momenta_unc_x,pion_momenta_1_unc_x,pion_momenta_2_unc_x)
py_unc = quadrature(kaon_momenta_unc_y,pion_momenta_1_unc_y,pion_momenta_2_unc_y)
pz_unc = quadrature(kaon_momenta_unc_z,pion_momenta_1_unc_z,pion_momenta_2_unc_z)

def find_invariant_mass_unc(E1,E2,E3,x,y,z,x_u,y_u,z_u,E1_u,E2_u,E3_u):
    sum_energy = (E1+E2+E3)
    energy_part = (sum_energy*E1_u)**2 + (sum_energy*E2_u)**2 + (sum_energy*E3_u)**2
    momentum_part = (-1*x*x_u)**2 + (-1*y*y_u)**2 + (-1*z*z_u)**2
    return np.sqrt(energy_part+ momentum_part)


invariant_masses_array_B_pm = find_invariant_mass_new(kaon_energies,
                                                        pion_1_energies,pion_2_energies,
                                                 (kaon_momenta_x+pion_momenta_1_x+pion_momenta_2_x),
                                                 (kaon_momenta_y+pion_momenta_1_y+pion_momenta_2_y),
                                                 (kaon_momenta_z+pion_momenta_1_z+pion_momenta_2_z))

invariant_masses_array_B_pm_unc = find_invariant_mass_unc(kaon_energies,pion_1_energies,pion_2_energies,
                                                 (kaon_momenta_x+pion_momenta_1_x+pion_momenta_2_x),
                                                 (kaon_momenta_y+pion_momenta_1_y+pion_momenta_2_y),
                                                 (kaon_momenta_z+pion_momenta_1_z+pion_momenta_2_z),
                                                 px_unc,py_unc,pz_unc,
                                                 kaon_energies_unc,pion_1_energies_unc,pion_2_energies_unc)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 6))
#fig.subplots_adjust(wspace=0.3) # increase horizontal space between plots

# This plots two 1D-histograms.
# The color is changed automatically, the styles are set by hand
# keep hold of the pT histogram data for fitting later
num_bins = 500
data_B_pm_hist = ax.hist(invariant_masses_array_B_pm, bins = num_bins, range = [4000,6500],histtype='step',label='$Entries$',color = 'black')
ax.grid(True,ls=':')
bin_width = (6500-4000)/num_bins
ax.set_xlabel('Invariant Mass of The B Meson (MeV)')
ax.set_ylabel(f'Entries per {round(bin_width)} MeV')
ax.set_title(r'Three-Body Invariant Mass Distribution for $B^\pm$')
ax.legend()
plt.show()

The above data shows all events. The B$^{\pm}$ decay is governed by the kaon charge. So, we can simply filter for K$^{\pm}$ to get the corresponding decay histograms.

In [ ]:
invariant_masses_array_B_plus = invariant_masses_array_B_pm[kaon_charges==1]
invariant_masses_array_B_minus = invariant_masses_array_B_pm[kaon_charges==-1]

In [ ]:
y = data_B_pm_hist[0]
#y_new = datahist_new[0]
x = np.delete(data_B_pm_hist[1],-1)
x_new = data_B_pm_hist[1]
#x_newnew = datahist_new[1]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 6))
#fig.subplots_adjust(wspace=0.3) # increase horizontal space between plots
# This plots two 1D-histograms.
# The color is changed automatically, the styles are set by hand
# keep hold of the pT histogram data for fitting later
num_bins = 500
data_B_minus_hist = ax.hist(invariant_masses_array_B_minus, bins = num_bins, range = [4000,6500],histtype='step',label='$B^- Entries$',color = 'black')
data_B_plus_hist = ax.hist(invariant_masses_array_B_plus, bins = num_bins, range = [4000,6500],histtype='step',label='$B^+ Entries$',color = 'c')
ax.grid(True,ls=':')
bin_width = (6500-4000)/num_bins
ax.set_xlabel('Invariant Mass of The B Meson (MeV)')
ax.set_ylabel(f'Entries per {round(bin_width)} MeV')
ax.set_title(r'Three-Body Invariant Mass Distribution for $B^+$ and $B^-$')
ax.legend()
plt.show()

In [ ]:
y_b_minus = data_B_minus_hist[0]
x_b_minus = data_B_minus_hist[1]
x_b_minus = np.delete(x_b_minus,-1)

y_b_plus = data_B_plus_hist[0]
x_b_plus = data_B_plus_hist[1]
x_b_plus = np.delete(x_b_plus,-1)



#This mask does nothing to the data, it simply helps us to filter the histogram plots so they only involve interactions we care about.
mask = (invariant_masses_array_B_pm>5200) & (invariant_masses_array_B_pm<5350)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(7, 3))

ax.hist(invariant_masses_array_B_pm[mask], bins = num_bins, range = [4000,6500],histtype='step',label='$Entries$',color = 'black')
plt.show()

## Resonances

The $R^0$ can decay by $K^+\pi^-$ or $\pi^+\pi^-$. So let us plot two body decays for this.

In [ ]:
def two_body_invariant_mass(H1_energies,H2_energies,X_momenta,Y_momenta,Z_momenta):
    
    invariant_masses = (np.sqrt(np.square(H1_energies+H2_energies) - 
                                (np.square(X_momenta)+np.square(Y_momenta)+np.square(Z_momenta))))
    return invariant_masses

R0_1_inv_mass = two_body_invariant_mass(kaon_energies,pion_1_energies,
                                        (kaon_momenta_x+pion_momenta_1_x),
                                        (kaon_momenta_y+pion_momenta_1_y),
                                        (kaon_momenta_z+pion_momenta_1_z))

R0_2_inv_mass = two_body_invariant_mass(pion_1_energies,pion_2_energies,
                                        (pion_momenta_1_x+pion_momenta_2_x),
                                        (pion_momenta_1_y+pion_momenta_2_y),
                                        (pion_momenta_1_z+pion_momenta_2_z))
R0_3_inv_mass = two_body_invariant_mass(kaon_energies,pion_2_energies,
                                        (kaon_momenta_x+pion_momenta_2_x),
                                        (kaon_momenta_y+pion_momenta_2_y),
                                        (kaon_momenta_z+pion_momenta_2_z))

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 7))

hist1 = ax.hist(R0_1_inv_mass[mask], bins=num_bins,  color='black', alpha=0.5, label='K,$\pi^+$')

ax.hist(R0_2_inv_mass[mask], bins=num_bins, color='c',alpha = 0.4 , label='$\pi^+,\pi^-$')

ax.hist(R0_3_inv_mass[mask],bins = num_bins,  color='gray', alpha=0.5, label='K,$\pi^-$')

bin_edges = hist1[1]

bin_width = bin_edges[1] - bin_edges[0]

m_D0 = 1864.8
ax.axvline(m_D0, color='red', linestyle='-',label = r'$M_{D^0}$')
ax.axvline(3096.9, color='green', linestyle='-',label = r'$M_{J\psi}$')

ax.grid(True, ls='--')
ax.set_xlabel('Invariant Mass of The Two Body System (MeV)')
ax.set_ylabel(f'Entries per {round(bin_width)} MeV')
ax.set_title(r'Invariant Mass Distribution of Two Body Systems')
ax.legend()
plt.show()

In [ ]:
D0_mass_lower = m_D0-30  # lower bound of D0 mass range to remove (in MeV/c^2)
D0_mass_upper = m_D0+30  # upper bound of D0 mass range to remove (in MeV/c^2)

# Create a mask to filter out the D0 resonance
remove_D0_resonances = ((R0_1_inv_mass < D0_mass_lower) | (R0_1_inv_mass > D0_mass_upper)) & ((R0_3_inv_mass < D0_mass_lower) | (R0_3_inv_mass > D0_mass_upper))

# Apply the mask to remove the D0 resonance from the original data
R0_1_inv_mass_filtered = R0_1_inv_mass[remove_D0_resonances]
R0_2_inv_mass_filtered = R0_2_inv_mass[remove_D0_resonances]
R0_3_inv_mass_filtered = R0_3_inv_mass[remove_D0_resonances]
mask_no_D0 = mask[remove_D0_resonances]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 7))

hist1 = ax.hist(R0_1_inv_mass_filtered[mask_no_D0], bins=num_bins,  color='black', alpha=0.5, label=r'K,$\pi^+$')
hist2 = ax.hist(R0_3_inv_mass_filtered[mask_no_D0], bins=num_bins,  color='gray', alpha=0.5, label=r'K,$\pi^-$')

ax.hist(R0_2_inv_mass_filtered[mask_no_D0], bins=num_bins, color='c',alpha = 0.4 , label='$\pi^{+},\pi^-$')

ax.axvline(m_D0, color='blue', linestyle='-',label = r'$M_{D^0}$')
ax.axvline(3096.9, color='green', linestyle='-',label = r'$M_{J\psi}$')

bin_edges = hist1[1]

bin_width = bin_edges[1] - bin_edges[0]

ax.grid(True, ls='--')
ax.set_xlabel('Invariant Mass of The Two Body System (MeV)')
ax.set_ylabel(f'Entries per {round(bin_width)} MeV')
ax.set_title(r'Invariant Mass Distribution of Two Body Systems (With $D^0$ Resonance Peak Removed)')
ax.legend()
plt.show()

In [ ]:
invariant_masses_array_B_pm_no_D0_resonance = invariant_masses_array_B_pm[remove_D0_resonances]
kaon_charges_no_D0_resonance = kaon_charges[remove_D0_resonances]
invariant_masses_array_B_plus_no_D0_resonance = invariant_masses_array_B_pm_no_D0_resonance[kaon_charges_no_D0_resonance==1]
invariant_masses_array_B_minus_no_D0_resonance = invariant_masses_array_B_pm_no_D0_resonance[kaon_charges_no_D0_resonance==-1]

In [ ]:
# plot the two body invariant mass for K,pi as if it is a K,K decay
kaon_2_energies= find_individual_energies(pion_2_p_abs, kaon_rest_mass_mev)
kaon_3_energies= find_individual_energies(pion_1_p_abs, kaon_rest_mass_mev)

D_KK_inv_mass = two_body_invariant_mass(kaon_energies,kaon_2_energies,
                                        (kaon_momenta_x+pion_momenta_2_x),
                                        (kaon_momenta_y+pion_momenta_2_y),
                                        (kaon_momenta_z+pion_momenta_2_z))

D_KK_inv_mass_2 = two_body_invariant_mass(kaon_energies,kaon_3_energies,
                                        (kaon_momenta_x+pion_momenta_1_x),
                                        (kaon_momenta_y+pion_momenta_1_y),
                                        (kaon_momenta_z+pion_momenta_1_z))


D_KK_inv_mass_2 = D_KK_inv_mass_2[remove_D0_resonances]
D_KK_inv_mass = D_KK_inv_mass[remove_D0_resonances]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 6))

hist1 = ax.hist(D_KK_inv_mass[mask_no_D0], bins = num_bins,histtype = 'step',
                          color = 'c',label = 'k,')

ax.hist(D_KK_inv_mass_2[mask_no_D0], bins = num_bins,histtype = 'step',
                          color = 'r',label = 'Dominant Final State 2 (plus)')
bin_edges = hist1[1]

bin_width = bin_edges[1] - bin_edges[0]

#ax.axvline(m_D0+100, color='green', linestyle='-')
#ax.axvline(m_D0+80, color='blue', linestyle='-')
#ax.axvline(m_D0+120, color='blue', linestyle='-')

#ax.axvline(m_D0, color='red', linestyle='-',label = r'$M_{D^0}$')
#ax.axvline(m_D0+40, color='k', linestyle='-',label = 'Lower Bound')
#ax.axvline(m_D0-40, color='k', linestyle='-',label = 'Upper Bound')


ax.legend()
ax.grid(True,ls=':')
ax.set_xlabel('Invariant Mass of The Resonance (MeV)')
ax.set_ylabel(f'Entries per {round(bin_width)} MeV')
ax.set_title('')
ax.legend()
plt.show()

The D$^0$ Meson has a known mass, and a known Resonance Width $\Gamma$, which is governed by the relation 
$$ \tau = \frac{\hbar}{\Gamma}.$$ PDGLive quotes the $\tau$, the mean life, of the D Meson. So the veto is applied over the width.

The resonance width comes from the uncertainty relation's energy form:
$$ \Delta E \Delta t \geq \hbar$$ 
for a resonance, which is where a decay becomes responsive at a certain energy (in our case it's the D$^{0} \rightarrow $K$^{+}$ K$^{-}$, and the energy is the Known D$^{0}$ mass). So the $\Delta E$ is simply just the "uncertainty on the energy" which in this case is the Resonance Width $\Gamma$. The $\Delta t = \tau$ and so the relation between $\Gamma \; \& \;\tau$ is regained.

In [ ]:
D0_tau = 4.103e-13
h_bar = 6.582e-16 #eV/s
D0_Gamma = h_bar/D0_tau *1e-6 #convert to MeV

#D_KK_inv_mass_2

lower_KK_filter =m_D0-40
higher_KK_filter = m_D0+40
remove_KK_resonances = ((D_KK_inv_mass < lower_KK_filter) | (D_KK_inv_mass > higher_KK_filter)) & ((D_KK_inv_mass_2< m_D0-40)|(D_KK_inv_mass_2> m_D0+40))
D_KK_inv_mass_filtered = D_KK_inv_mass[remove_KK_resonances]
D_KK_inv_mass_2_filtered = D_KK_inv_mass_2[remove_KK_resonances]
mask_no_KK = mask_no_D0[remove_KK_resonances]

R0_1_inv_mass_filtered_no_kk = R0_1_inv_mass_filtered[remove_KK_resonances]
R0_2_inv_mass_filtered_no_kk = R0_2_inv_mass_filtered[remove_KK_resonances]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 7))

hist1 = ax.hist(D_KK_inv_mass_filtered[mask_no_KK], bins=num_bins, histtype = 'step',  color='c', alpha=0.5, label=r'K,K')
ax.hist(D_KK_inv_mass_2_filtered[mask_no_KK], bins = num_bins,histtype = 'step',
                          color = 'r',label = 'Dominant Final State 2')

ax.axvline(1864.8, color='green', linestyle='-',label = r'$M_{D^0}$')
bin_edges = hist1[1]

bin_width = bin_edges[1] - bin_edges[0]


ax.grid(True, ls='--')
ax.set_xlabel('Invariant Mass of The 2 Body Decay (MeV)')
ax.set_ylabel(f'Entries per {round(bin_width)} MeV')
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 7))

hist1 = ax.hist(R0_1_inv_mass_filtered_no_kk[mask_no_KK], bins=num_bins,  color='black', alpha=0.5, label=r'K,\pi')
ax.hist(R0_2_inv_mass_filtered_no_kk[mask_no_KK], bins=num_bins, color='c',alpha = 0.4 , label='$\pi^{+},\pi$')
bin_edges = hist1[1]

bin_width = bin_edges[1] - bin_edges[0]

ax.axvline(m_D0, color='blue', linestyle='-',label = r'$M_{D^0}$')
ax.axvline(3096.9, color='green', linestyle='-',label = r'$M_{J\psi}$')

ax.grid(True, ls=':')
ax.set_xlabel('Invariant Mass of The 2 Body Decay (MeV)')
ax.set_ylabel(f'Entries per {int(bin_width)} MeV')
ax.legend()
plt.show()
# Extract the bin edges from hist1


In [ ]:
kaon_charges_no_resonances = kaon_charges_no_D0_resonance[remove_KK_resonances]
invariant_masses_array_B_pm_no_resonances = invariant_masses_array_B_pm_no_D0_resonance[remove_KK_resonances]
invariant_masses_array_B_plus_no_resonances = invariant_masses_array_B_pm_no_resonances[kaon_charges_no_resonances==1]
invariant_masses_array_B_minus_no_resonances = invariant_masses_array_B_pm_no_resonances[kaon_charges_no_resonances==-1]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 6))
#fig.subplots_adjust(wspace=0.3) # increase horizontal space between plots

# This plots two 1D-histograms.
# The color is changed automatically, the styles are set by hand
# keep hold of the pT histogram data for fitting later

num_bins = 500

data_B_plus_hist = ax.hist(invariant_masses_array_B_plus_no_resonances, bins = num_bins, range = [4000,6500],histtype = 'step',
                          color = 'c',label = '$B^{+}$')
data_B_minus_hist = ax.hist(invariant_masses_array_B_minus_no_resonances, bins = num_bins, range = [4000,6500],histtype = 'step',
                           color = 'black',label = '$B^{-}$')
ax.axvline(5280, color='green', linestyle='-',label = r'$M_{B^+}$')
ax.legend()
ax.grid(True,ls=':')
bin_width = (6500- 4000)/num_bins
ax.set_xlabel('Invariant Mass of The B Meson (MeV)')
ax.set_ylabel(f'Entries per {int(bin_width)} MeV')
ax.set_title('Invariant Mass Distribution of $B^+$ and $B^-$ Mesons')
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 6))
#fig.subplots_adjust(wspace=0.3) # increase horizontal space between plots

# This plots two 1D-histograms.
# The color is changed automatically, the styles are set by hand
# keep hold of the pT histogram data for fitting later


data_B_pm_hist = ax.hist(invariant_masses_array_B_pm_no_resonances, bins = num_bins, range = [4000,6500],histtype = 'step',
                          color = 'c',label = '$B^{+}$')

ax.legend()
ax.grid(True,ls=':')
ax.set_xlabel('Invariant Mass of The B Meson (MeV)')
ax.set_ylabel(f'Entries per {round(bin_width)} MeV')
ax.set_title('Invariant Mass Distribution of $B^+$ and $B^-$ Mesons')
ax.legend()
plt.show()

In [ ]:
y_b_minus = data_B_minus_hist[0]
x_b_minus = data_B_minus_hist[1]
x_b_minus = np.delete(x_b_minus,-1)

y_b_plus = data_B_plus_hist[0]
x_b_plus = data_B_plus_hist[1]
x_b_plus = np.delete(x_b_plus,-1)

y_b_pm = data_B_pm_hist[0]
x_b_pm = data_B_pm_hist[1]
x_b_pm = np.delete(x_b_pm,-1)

### 5.5 Global CP Asymmetry

In [ ]:
start_val = 5100
fixed_gaussian_peak = 5100
max_val = 5600

In [ ]:
mask_fit = (x_b_pm>start_val) & (x_b_pm<max_val)
x_b_pm_new = x_b_pm[mask_fit]
y_b_pm_new = y_b_pm[mask_fit]
y_b_pm_new = np.delete(y_b_pm_new,np.argmin(x_b_pm_new))

In [ ]:
mask_fit = (x_b_minus>start_val) & (x_b_minus<max_val)
x_b_minus_new = x_b_minus[mask_fit]
y_b_minus_new = y_b_minus[mask_fit]
y_b_minus_new = np.delete(y_b_minus_new,np.argmin(x_b_minus_new))

In [ ]:
mask_fit = (x_b_plus>start_val) & (x_b_plus<max_val)
x_b_plus_new = x_b_plus[mask_fit]
y_b_plus_new = y_b_plus[mask_fit]
y_b_plus_new = np.delete(y_b_plus_new,np.argmin(x_b_plus_new))

In [ ]:
beta = 1.5
from scipy.stats import crystalball
from scipy.stats import norm


def exponential(x, normE, decay):
    xoffset = 5000 # this is a technical parameter, which can be used to move the position at which the function evaluates to "norm"
    
    return np.array( normE * np.exp(-(x-xoffset)/(decay)) )


def crystal_ball_scipy(x, m, mean, sigma, amp):
    # Normalize the Crystal Ball function to use in the fit.
    # The 'norm' parameter scales the function to the data.
    return amp * crystalball.pdf(x, beta, m, loc=mean, scale=sigma)



def gaussian(x, amplitude, mean, stddev):
    xoffset = 0  # Shift position
    return   np.array(amplitude * (1/np.sqrt(2*np.pi*stddev**2)) *np.exp(-((x - mean - xoffset) ** 2) / (2 * stddev ** 2)))

def fit_function(x,normE,decay, m, meanCB, sigmaCB, normCB):
    return np.array(crystal_ball_scipy(x, m, meanCB, sigmaCB, normCB) +  gaussian(x, fixed_gaussian_amp, fixed_gaussian_peak, fixed_gaussian_std)+ exponential(x, normE, decay))

def fit_data(bins, values, minX, maxX, p0, values_unc):
    # determine bin centres
    bin_centres = [(a+b)/2 for a,b in zip(bins[0:-1],bins[1:]) ] # uses simultaneous loop over two arrays

    # reduce range to fit only part of curve
    bin_centres_red = [] 
    values_red = []
    for c,v in zip(bin_centres,values):
        if c < minX or c > maxX: continue
        bin_centres_red.append(c)
        values_red.append(v)

        
    # execute the fit with starting values as given in p0
    bin_centres_red = np.array(bin_centres_red)
    values_red = np.array(values_red)

    coeff_fit,cov_fit = curve_fit(fit_function,bin_centres_red,values_red,p0,sigma=values_unc,maxfev = 1000000) # fit
    
    # evaluate chi2
    fit_vals = [fit_function(x,coeff_fit[0],coeff_fit[1],coeff_fit[2]
                             ,coeff_fit[3],coeff_fit[4],coeff_fit[5]) for x in bin_centres_red]
    chi2parts = np.array( ( np.divide( np.array(values_red) - np.array(fit_vals), np.sqrt( values_red ), 
                                      out = np.array(values_red), where = np.array(values_red) != 0 ) )**2 )
    chi2 = np.sum( chi2parts )
    
    return coeff_fit,cov_fit, bin_centres, bin_centres_red, chi2, len(chi2parts)


def print_results(coeff,cov,chi2,ndf):
    perr = np.sqrt(np.diag(cov)) # extract errors from covarianve matrix
    # output fit results
    print('Fit results with chi2/ndf', chi2,'/',ndf)
    print('Reduced Chi Squared = ', chi2/ndf)
    parcount = 0
    for p,e in zip(coeff,perr):
        parcount += 1
        print('Par {:d}: {:f} +/- {:f}'.format(parcount,p,e))

def plot_results(ax,bin_centres,bin_centres_red,values,coeff_fit,fname, plus_or_minus):
  # plot the data, this time as dots with error bars (sqrt(N) errors)
    
    ax.errorbar(bin_centres_red,values,yerr=np.sqrt(values),linestyle='',marker='.',markerfacecolor='k',markeredgecolor='k',ecolor='k',label='Data')
    fit_x_vals = np.linspace(bin_centres_red[0],bin_centres_red[-1],1000)
  # plot the fit: create x values, then calculate the corresponding y values and plot
    y_fit = [fit_function(x,coeff_fit[0],coeff_fit[1],coeff_fit[2]
                             ,coeff_fit[3],coeff_fit[4],coeff_fit[5]) for x in fit_x_vals]
    ax.plot(fit_x_vals,y_fit,label='Fit',color='r',zorder=10) # zorder makes sure the fit line is on top
    ax.plot(fit_x_vals,exponential(fit_x_vals,coeff_fit[0],coeff_fit[1]),color = 'g',label = 'Exponential Function')
    ax.plot(fit_x_vals, crystal_ball_scipy(fit_x_vals,coeff_fit[2],coeff_fit[3],coeff_fit[4],coeff_fit[5]), color='c', label='Crystal Ball Function')
    ax.plot(fit_x_vals, gaussian(fit_x_vals, fixed_gaussian_amp, fixed_gaussian_peak, fixed_gaussian_std), color='y', label='Gaussian Function')
    ax.axvline(coeff_fit[3]+2*coeff_fit[4], color='green', linestyle='-')
    ax.axvline(coeff_fit[3]-2*coeff_fit[4], color='green', linestyle='-')

    ax.legend()
    ax.set_xlabel('$m_{inv}$ in MeV')
    ax.set_ylabel(f'Candidates per {bin_width} MeV')
    ax.set_title(f'Invariant Mass Distribution Fit $B^{plus_or_minus}$')
    #plt.savefig(fname)
    
    return np.array([coeff_fit[2],coeff_fit[3],coeff_fit[4],coeff_fit[5]]),np.array([coeff_fit[0],coeff_fit[1]])

#gaussian_data_minus in order [amplitude, mean, stddev]
#exponential_data_minus in order [normE,decay]

In [ ]:
p0 = [3250, 1500,40, 5280, 2*10, 1000000]
fixed_gaussian_peak = start_val
fixed_gaussian_std = 30
fixed_gaussian_amp = 2*26000

sigma = np.sqrt(np.array(y_b_pm_new))

coeff_pT,cov_pT, bin_centres_pT, bin_centres_red_pT, chi2_pT, ndf_pT = fit_data(x_b_pm_new, y_b_pm_new, start_val, max_val, 
                                                        p0, sigma)
                                                        #normE, decay, amplitude1,mean1, stddev1


#print(x_new[y.argmax()])
print_results(coeff_pT,cov_pT, chi2_pT, ndf_pT)
#
# plot results
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 6), dpi=1000)
gaussian_data_pm,exponential_data_pm = plot_results(ax,bin_centres_pT,bin_centres_red_pT,y_b_pm_new,coeff_pT,'fit_pm.pdf', '±')



#normE,decay, m, meanCB, sigmaCB, normCB

In [ ]:
fixed_m = gaussian_data_pm[0]
fixed_sigma = gaussian_data_pm[2]

def crystal_ball_scipy(x, mean, amp):
    # Normalize the Crystal Ball function to use in the fit.
    # The 'norm' parameter scales the function to the data.
    return amp * crystalball.pdf(x, beta, fixed_m, loc=mean, scale=fixed_sigma)



def fit_function(x,normE,decay, meanCB, normCB):
    return np.array(crystal_ball_scipy(x, meanCB, normCB) +  gaussian(x, fixed_gaussian_amp, fixed_gaussian_peak, fixed_gaussian_std)+ exponential(x, normE, decay))


In [ ]:
def plot_results(ax,bin_centres,bin_centres_red,values,coeff_fit,fname, plus_or_minus):
  # plot the data, this time as dots with error bars (sqrt(N) errors)
    
    ax.errorbar(bin_centres_red,values,yerr=np.sqrt(values),linestyle='',marker='.',markerfacecolor='k',markeredgecolor='k',ecolor='k',label='Data')
    fit_x_vals = np.linspace(bin_centres_red[0],bin_centres_red[-1],1000)
  # plot the fit: create x values, then calculate the corresponding y values and plot
    y_fit = [fit_function(x,coeff_fit[0],coeff_fit[1],coeff_fit[2]
                             ,coeff_fit[3]) for x in fit_x_vals]
    ax.plot(fit_x_vals,y_fit,label='Fit',color='r',zorder=10) # zorder makes sure the fit line is on top
    ax.plot(fit_x_vals,exponential(fit_x_vals,coeff_fit[0],coeff_fit[1]),color = 'g',label = 'Exponential Function')
    ax.plot(fit_x_vals, crystal_ball_scipy(fit_x_vals,coeff_fit[2],coeff_fit[3]), color='c', label='Crystal Ball Function')
    ax.plot(fit_x_vals, gaussian(fit_x_vals, fixed_gaussian_amp, fixed_gaussian_peak, fixed_gaussian_std), color='y', label='Gaussian Function')
    ax.axvline(coeff_fit[2]+2*fixed_sigma, color='green', linestyle='-')
    ax.axvline(coeff_fit[2]-2*fixed_sigma, color='green', linestyle='-')

    ax.legend()
    ax.set_xlabel('$m_{inv}$ in MeV')
    ax.set_ylabel(f'Candidates per {bin_width} MeV')
    #ax.set_title(f'Invariant Mass Distribution Fit $B^{plus_or_minus}$')
    #plt.savefig(fname)
    
    return np.array([coeff_fit[2],coeff_fit[3]]),np.array([coeff_fit[0],coeff_fit[1]])

In [ ]:
def fit_data(bins, values, minX, maxX, p0, values_unc):
    # determine bin centres
    bin_centres = [(a+b)/2 for a,b in zip(bins[0:-1],bins[1:]) ] # uses simultaneous loop over two arrays

    # reduce range to fit only part of curve
    bin_centres_red = [] 
    values_red = []
    for c,v in zip(bin_centres,values):
        if c < minX or c > maxX: continue
        bin_centres_red.append(c)
        values_red.append(v)

    bin_centres_red = np.array(bin_centres_red)
    values_red = np.array(values_red)
    # execute the fit with starting values as given in p0
    coeff_fit,cov_fit = curve_fit(fit_function,bin_centres_red,values_red,p0,sigma=values_unc,maxfev = 1000000) # fit
    
    # evaluate chi2
    fit_vals = [fit_function(x,coeff_fit[0],coeff_fit[1],coeff_fit[2]
                             ,coeff_fit[3]) for x in bin_centres_red]
    chi2parts = np.array( ( np.divide( np.array(values_red) - np.array(fit_vals), np.sqrt( values_red ), 
                                      out = np.array(values_red), where = np.array(values_red) != 0 ) )**2 )
    chi2 = np.sum( chi2parts )
    
    return coeff_fit,cov_fit, bin_centres, bin_centres_red, chi2, len(chi2parts)

In [ ]:
p0 = [200, 1500, 5280, 1000000]

#normE,decay, meanCB, normCB

In [ ]:
fixed_gaussian_std = 30
fixed_gaussian_amp = 26000

sigma = np.sqrt(np.array(y_b_minus_new))
coeff_pT,cov_pT, bin_centres_pT_minus, bin_centres_red_pT_minus, chi2_pT, ndf_pT = fit_data(x_b_minus_new, y_b_minus_new, start_val, max_val, 
                                                        p0, sigma)
                                                        #normE, decay, amplitude1,mean1, stddev1


#print(x_new[y.argmax()])
print_results(coeff_pT,cov_pT, chi2_pT, ndf_pT)
#
# plot results
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 6), dpi=1000)
gaussian_data_minus,exponential_data_minus = plot_results(ax,bin_centres_pT_minus,bin_centres_red_pT,y_b_minus_new,coeff_pT,'fit_pT.pdf', '-')


#normE,decay, m, meanCB, sigmaCB, normCB

In [ ]:
from scipy.integrate import quad

def integrate_exponential(a, b, normE, decay):
    result, error = quad(exponential, a, b,args = (normE,decay))
    return result



exp_count_minus = np.round(integrate_exponential(gaussian_data_minus[0]-2.0*fixed_sigma,
                                                 gaussian_data_minus[0]+2.0*fixed_sigma, 
                                                 exponential_data_minus[0], exponential_data_minus[1]), decimals=0)

print(exp_count_minus)


In [ ]:
x_values = np.linspace(gaussian_data_minus[0] - 2 * fixed_sigma, 
                       gaussian_data_minus[0] + 2 * fixed_sigma, 1000)
y_values = exponential(x_values, exponential_data_minus[0], exponential_data_minus[1])

plt.fill_between(x_values, y_values, where=((x_values >= gaussian_data_minus[0] - 2.05 * fixed_sigma) & 
                                            (x_values <= gaussian_data_minus[0] + 2.05 * fixed_sigma)), 
                 color="red", alpha=0.3, label=f'Integrated Area = {exp_count_minus}')
plt.plot(x_values, y_values, label='Exponential Function')
plt.legend()
plt.xlabel('x')
plt.ylabel('Exponential(x)')
plt.title('Exponential Function Integration')
plt.grid(True)
plt.show()

In [ ]:
'''def fit_data(bins, values, minX, maxX, p0):
    # determine bin centres
    bin_centres = [(a+b)/2 for a,b in zip(bins[0:-1],bins[1:]) ] # uses simultaneous loop over two arrays

    # reduce range to fit only part of curve
    bin_centres_red = [] 
    values_red = []
    for c,v in zip(bin_centres,values):
        if c < minX or c > maxX: continue
        bin_centres_red.append(c)
        values_red.append(v)

    bin_centres_red = np.array(bin_centres_red)
    values_red = np.array(values_red)
    # execute the fit with starting values as given in p0
    coeff_fit,cov_fit = curve_fit(fit_function,bin_centres_red,values_red,p0,maxfev = 1000000) # fit
    
    # evaluate chi2
    fit_vals = [fit_function(x,coeff_fit[0],coeff_fit[1],coeff_fit[2]
                             ,coeff_fit[3]) for x in bin_centres_red]
    chi2parts = np.array( ( np.divide( np.array(values_red) - np.array(fit_vals), np.sqrt( values_red ), 
                                      out = np.array(values_red), where = np.array(values_red) != 0 ) )**2 )
    chi2 = np.sum( chi2parts )
    
    return coeff_fit,cov_fit, bin_centres, bin_centres_red, chi2, len(chi2parts)'''

fixed_gaussian_std = 30
fixed_gaussian_amp = 24000
#normE,decay, m, meanCB, sigmaCB, normCB

sigma = np.sqrt(np.array(y_b_plus_new))

coeff_pT,cov_pT, bin_centres_pT_plus, bin_centres_red_pT, chi2_pT, ndf_pT = fit_data(x_b_plus_new, y_b_plus_new, start_val, max_val, 
                                                        p0, sigma)
                                                        #normE, decay, amplitude1,mean1, stddev1


#print(x_new[y.argmax()])
print_results(coeff_pT,cov_pT, chi2_pT, ndf_pT)

# plot results
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 6), dpi=1000)
gaussian_data_plus,exponential_data_plus = plot_results(ax,bin_centres_pT_plus,bin_centres_red_pT,y_b_plus_new,coeff_pT,'fit_pT.pdf', '+')


#normE,decay, m, meanCB, sigmaCB, normCB

In [ ]:
exp_count_plus = np.round(integrate_exponential(gaussian_data_plus[0]-2.0*fixed_sigma,
                                                gaussian_data_plus[0]+2.0*fixed_sigma, 
                                                exponential_data_plus[0], exponential_data_plus[1]), decimals=0)

print(exp_count_plus)

In [ ]:
from scipy.integrate import quad

N_minus, N_minus_error = quad(crystal_ball_scipy, 0, np.inf, args=(gaussian_data_minus[0], gaussian_data_minus[1]))
N_plus, N_plus_error = quad(crystal_ball_scipy, 0, np.inf, args=(gaussian_data_plus[0], gaussian_data_plus[1]))

print("N_minus:", N_minus,'±',N_minus_error)
print("N_plus:", N_plus,'±',N_plus_error)

In [ ]:
# The CP Asymmetry is
A = (N_minus - N_plus)/(N_minus + N_plus)
error_A = np.sqrt((1-A**2)/(N_plus+N_minus))

print("A:", A,'±',error_A)

### Resonance Removal

In [ ]:
kaon_energies_no_resonances =  kaon_energies[remove_D0_resonances][remove_KK_resonances]
pion_1_energies_no_resonances =  pion_1_energies[remove_D0_resonances][remove_KK_resonances]
pion_2_energies_no_resonances =  pion_2_energies[remove_D0_resonances][remove_KK_resonances]

kaon_momenta_x_no_resonances =  kaon_momenta_x[remove_D0_resonances][remove_KK_resonances]
kaon_momenta_y_no_resonances =  kaon_momenta_y[remove_D0_resonances][remove_KK_resonances]
kaon_momenta_z_no_resonances =  kaon_momenta_z[remove_D0_resonances][remove_KK_resonances]

pion_momenta_1_x_no_resonances =  pion_momenta_1_x[remove_D0_resonances][remove_KK_resonances]
pion_momenta_1_y_no_resonances =  pion_momenta_1_y[remove_D0_resonances][remove_KK_resonances]
pion_momenta_1_z_no_resonances =  pion_momenta_1_z[remove_D0_resonances][remove_KK_resonances]

pion_momenta_2_x_no_resonances =  pion_momenta_2_x[remove_D0_resonances][remove_KK_resonances]
pion_momenta_2_y_no_resonances =  pion_momenta_2_y[remove_D0_resonances][remove_KK_resonances]
pion_momenta_2_z_no_resonances =  pion_momenta_2_z[remove_D0_resonances][remove_KK_resonances]

kaon_charges_no_resonances =  kaon_charges[remove_D0_resonances][remove_KK_resonances]

### Dalitz Plotting

Let us define variables that let us choose events of interest for the Signal region in the Dalitz Plot. We will choose a range of:
$$\mu \pm 2\sigma$$ 
from the above plot in order to select the region of interest

In [ ]:
num_bins_x = 100
num_bins_y = 100

In [ ]:
B_minus_mean,B_minus_std = gaussian_data_minus[0], fixed_sigma
B_plus_mean,B_plus_std = gaussian_data_plus[0], fixed_sigma

In [ ]:
center_gaussian_mask = (invariant_masses_array_B_pm_no_resonances>B_minus_mean-2*B_minus_std) & (invariant_masses_array_B_pm_no_resonances<B_minus_mean+2*B_minus_std)
center_gaussian_kaon_energies = kaon_energies_no_resonances[center_gaussian_mask]

center_gaussian_pion_1_energies = pion_1_energies_no_resonances[center_gaussian_mask]

center_gaussian_pion_2_energies = pion_2_energies_no_resonances[center_gaussian_mask]

center_gaussian_kaon_momenta_x = kaon_momenta_x_no_resonances[center_gaussian_mask]

center_gaussian_kaon_momenta_y = kaon_momenta_y_no_resonances[center_gaussian_mask]

center_gaussian_kaon_momenta_z = kaon_momenta_z_no_resonances[center_gaussian_mask]

center_gaussian_pion_momenta_1_x = pion_momenta_1_x_no_resonances[center_gaussian_mask]

center_gaussian_pion_momenta_1_y = pion_momenta_1_y_no_resonances[center_gaussian_mask]

center_gaussian_pion_momenta_1_z = pion_momenta_1_z_no_resonances[center_gaussian_mask]

center_gaussian_pion_momenta_2_x = pion_momenta_2_x_no_resonances[center_gaussian_mask]
center_gaussian_pion_momenta_2_y = pion_momenta_2_y_no_resonances[center_gaussian_mask]
center_gaussian_pion_momenta_2_z = pion_momenta_2_z_no_resonances[center_gaussian_mask]

center_gaussian_kaon_charges = kaon_charges_no_resonances[center_gaussian_mask]

In [ ]:
center_gaussian_kaon_energies_B_plus = center_gaussian_kaon_energies[center_gaussian_kaon_charges == 1]

center_gaussian_pion_1_energies_B_plus = center_gaussian_pion_1_energies[center_gaussian_kaon_charges == 1]

center_gaussian_pion_2_energies_B_plus = center_gaussian_pion_2_energies[center_gaussian_kaon_charges == 1]

center_gaussian_kaon_momenta_x_B_plus = center_gaussian_kaon_momenta_x[center_gaussian_kaon_charges == 1]

center_gaussian_kaon_momenta_y_B_plus = center_gaussian_kaon_momenta_y[center_gaussian_kaon_charges == 1]

center_gaussian_kaon_momenta_z_B_plus = center_gaussian_kaon_momenta_z[center_gaussian_kaon_charges == 1]

center_gaussian_pion_momenta_1_x_B_plus = center_gaussian_pion_momenta_1_x[center_gaussian_kaon_charges == 1]

center_gaussian_pion_momenta_1_y_B_plus = center_gaussian_pion_momenta_1_y[center_gaussian_kaon_charges == 1]

center_gaussian_pion_momenta_1_z_B_plus = center_gaussian_pion_momenta_1_z[center_gaussian_kaon_charges == 1]

center_gaussian_pion_momenta_2_x_B_plus = center_gaussian_pion_momenta_2_x[center_gaussian_kaon_charges == 1]

center_gaussian_pion_momenta_2_y_B_plus = center_gaussian_pion_momenta_2_y[center_gaussian_kaon_charges == 1]

center_gaussian_pion_momenta_2_z_B_plus = center_gaussian_pion_momenta_2_z[center_gaussian_kaon_charges == 1]

#####

center_gaussian_kaon_energies_B_minus = center_gaussian_kaon_energies[center_gaussian_kaon_charges == -1]

center_gaussian_pion_1_energies_B_minus = center_gaussian_pion_1_energies[center_gaussian_kaon_charges == -1]

center_gaussian_pion_2_energies_B_minus = center_gaussian_pion_2_energies[center_gaussian_kaon_charges == -1]

center_gaussian_kaon_momenta_x_B_minus = center_gaussian_kaon_momenta_x[center_gaussian_kaon_charges == -1]

center_gaussian_kaon_momenta_y_B_minus = center_gaussian_kaon_momenta_y[center_gaussian_kaon_charges == -1]

center_gaussian_kaon_momenta_z_B_minus = center_gaussian_kaon_momenta_z[center_gaussian_kaon_charges == -1]

center_gaussian_pion_momenta_1_x_B_minus = center_gaussian_pion_momenta_1_x[center_gaussian_kaon_charges == -1]

center_gaussian_pion_momenta_1_y_B_minus = center_gaussian_pion_momenta_1_y[center_gaussian_kaon_charges == -1]

center_gaussian_pion_momenta_1_z_B_minus = center_gaussian_pion_momenta_1_z[center_gaussian_kaon_charges == -1]

center_gaussian_pion_momenta_2_x_B_minus = center_gaussian_pion_momenta_2_x[center_gaussian_kaon_charges == -1]

center_gaussian_pion_momenta_2_y_B_minus = center_gaussian_pion_momenta_2_y[center_gaussian_kaon_charges == -1]

center_gaussian_pion_momenta_2_z_B_minus = center_gaussian_pion_momenta_2_z[center_gaussian_kaon_charges == -1]

In [ ]:
K_Pi_1_inv_mass_B_plus = two_body_invariant_mass(center_gaussian_kaon_energies_B_plus, center_gaussian_pion_1_energies_B_plus,
                                          (center_gaussian_kaon_momenta_x_B_plus + center_gaussian_pion_momenta_1_x_B_plus),
                                          (center_gaussian_kaon_momenta_y_B_plus + center_gaussian_pion_momenta_1_y_B_plus),
                                          (center_gaussian_kaon_momenta_z_B_plus + center_gaussian_pion_momenta_1_z_B_plus))

K_Pi_2_inv_mass_B_plus = two_body_invariant_mass(center_gaussian_kaon_energies_B_plus, center_gaussian_pion_2_energies_B_plus,
                                          (center_gaussian_kaon_momenta_x_B_plus + center_gaussian_pion_momenta_2_x_B_plus),
                                          (center_gaussian_kaon_momenta_y_B_plus + center_gaussian_pion_momenta_2_y_B_plus),
                                          (center_gaussian_kaon_momenta_z_B_plus + center_gaussian_pion_momenta_2_z_B_plus))

Pi_Pi_inv_mass_B_plus = two_body_invariant_mass(center_gaussian_pion_1_energies_B_plus, center_gaussian_pion_2_energies_B_plus,
                                          (center_gaussian_pion_momenta_1_x_B_plus + center_gaussian_pion_momenta_2_x_B_plus),
                                          (center_gaussian_pion_momenta_1_y_B_plus + center_gaussian_pion_momenta_2_y_B_plus),
                                          (center_gaussian_pion_momenta_1_z_B_plus + center_gaussian_pion_momenta_2_z_B_plus))

print(len(K_Pi_1_inv_mass_B_plus))

# Calculate the squared invariant masses
m12_squared_B_plus = 1e-6 * np.square(K_Pi_1_inv_mass_B_plus) # k pi +
m13_squared_B_plus = 1e-6 * np.square(K_Pi_2_inv_mass_B_plus) # k pi - 
m23_squared_B_plus = 1e-6 * np.square(Pi_Pi_inv_mass_B_plus) # pi+ pi-

# Create the Dalitz plot as a heatmap
plt.figure(figsize=(3, 3), dpi=500)
hist1 = plt.hist2d(m23_squared_B_plus, m13_squared_B_plus, bins=num_bins_x, cmap='seismic')

plt.colorbar(hist1[3])

plt.xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
plt.ylabel('$m_{K^+\pi^-}^2$ (GeV$^2$/c$^4$)')
plt.suptitle('Dalitz Plot of $B^+$ Central Gaussian Region ($m_{K^+\pi^-}^2$ Against $m_{\pi^+,\pi^-}^2$)')
plt.tight_layout()
plt.grid(True)

plt.show()

In [ ]:
K_Pi_1_inv_mass_B_minus = two_body_invariant_mass(center_gaussian_kaon_energies_B_minus, center_gaussian_pion_1_energies_B_minus,
                                          (center_gaussian_kaon_momenta_x_B_minus + center_gaussian_pion_momenta_1_x_B_minus),
                                          (center_gaussian_kaon_momenta_y_B_minus + center_gaussian_pion_momenta_1_y_B_minus),
                                          (center_gaussian_kaon_momenta_z_B_minus + center_gaussian_pion_momenta_1_z_B_minus))

K_Pi_2_inv_mass_B_minus = two_body_invariant_mass(center_gaussian_kaon_energies_B_minus, center_gaussian_pion_2_energies_B_minus,
                                          (center_gaussian_kaon_momenta_x_B_minus + center_gaussian_pion_momenta_2_x_B_minus),
                                          (center_gaussian_kaon_momenta_y_B_minus + center_gaussian_pion_momenta_2_y_B_minus),
                                          (center_gaussian_kaon_momenta_z_B_minus + center_gaussian_pion_momenta_2_z_B_minus))

Pi_Pi_inv_mass_B_minus = two_body_invariant_mass(center_gaussian_pion_1_energies_B_minus, center_gaussian_pion_2_energies_B_minus,
                                          (center_gaussian_pion_momenta_1_x_B_minus + center_gaussian_pion_momenta_2_x_B_minus),
                                          (center_gaussian_pion_momenta_1_y_B_minus + center_gaussian_pion_momenta_2_y_B_minus),
                                          (center_gaussian_pion_momenta_1_z_B_minus + center_gaussian_pion_momenta_2_z_B_minus))


# Calculate the squared invariant masses
m12_squared_B_minus = 1e-6 * np.square(K_Pi_1_inv_mass_B_minus) # k pi +
m13_squared_B_minus = 1e-6 * np.square(K_Pi_2_inv_mass_B_minus) # k pi -
m23_squared_B_minus = 1e-6 * np.square(Pi_Pi_inv_mass_B_minus) # pi+ pi -

# Create the Dalitz plot as a heatmap
plt.figure(figsize=(3, 3), dpi=500)
hist2 = plt.hist2d(m23_squared_B_minus, m12_squared_B_minus, bins=num_bins_x, cmap='seismic')

plt.colorbar(hist1[3])

plt.xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
plt.ylabel('$m_{K^-\pi^+}^2$ (GeV$^2$/c$^4$)')
plt.suptitle('Dalitz Plot of $B^-$ Central Gaussian Region ($m_{K^-\pi^+}^2$ Against $m_{\pi^+,\pi^-}^2$)')
plt.tight_layout()
plt.grid(True)

plt.show()

```
# Finding the range to be considered 'background' so it can be subtracted

center_gaussian_mask_invariant_masses_array_B_pm = invariant_masses_array_B_pm[center_gaussian_mask]

center_gaussian_mask_invariant_masses_array_B_plus = center_gaussian_mask_invariant_masses_array_B_pm[center_gaussian_kaon_charges == 1]

B_plus_center_region_count = len(center_gaussian_mask_invariant_masses_array_B_plus)

center_gaussian_mask_invariant_masses_array_B_minus = center_gaussian_mask_invariant_masses_array_B_pm[center_gaussian_kaon_charges == -1]

B_minus_center_region_count = len(center_gaussian_mask_invariant_masses_array_B_minus)

###### constructing background mask:

lower_limit_background = 5350

background_lower_limit_plus_mask = (invariant_masses_array_B_plus>lower_limit_background)

unbounded_background_invariant_masses_array_B_plus = invariant_masses_array_B_plus[background_lower_limit_plus_mask]

background_invariant_masses_array_B_plus = unbounded_background_invariant_masses_array_B_plus[:int(exp_count_plus)]

upper_limit_B_plus_background = np.max(background_invariant_masses_array_B_plus)

background_lower_limit_minus_mask = (invariant_masses_array_B_minus>lower_limit_background)

background_invariant_masses_array_B_plus = unbounded_background_invariant_masses_array_B_plus[:int(exp_count_plus)]

unbounded_background_invariant_masses_array_B_minus = invariant_masses_array_B_minus[background_lower_limit_minus_mask]

background_invariant_masses_array_B_minus = unbounded_background_invariant_masses_array_B_minus[:int(exp_count_minus)]
``

In [ ]:
# Finding the range to be considered 'background' so it can be subtracted

center_gaussian_mask_invariant_masses_array_B_pm = invariant_masses_array_B_pm_no_resonances[center_gaussian_mask]

center_gaussian_mask_invariant_masses_array_B_plus = center_gaussian_mask_invariant_masses_array_B_pm[center_gaussian_kaon_charges == 1]

B_plus_center_region_count = len(center_gaussian_mask_invariant_masses_array_B_plus)

center_gaussian_mask_invariant_masses_array_B_minus = center_gaussian_mask_invariant_masses_array_B_pm[center_gaussian_kaon_charges == -1]

B_minus_center_region_count = len(center_gaussian_mask_invariant_masses_array_B_minus)

###### constructing background mask:

lower_limit_background = 5400

background_lower_limit_plus_mask = (invariant_masses_array_B_plus_no_resonances>lower_limit_background)

unbounded_background_invariant_masses_array_B_plus = invariant_masses_array_B_plus_no_resonances[background_lower_limit_plus_mask]

background_invariant_masses_array_B_plus = unbounded_background_invariant_masses_array_B_plus[:int(exp_count_plus)]

upper_limit_B_plus_background = np.max(background_invariant_masses_array_B_plus)



background_lower_limit_minus_mask = (invariant_masses_array_B_minus_no_resonances>lower_limit_background)

unbounded_background_invariant_masses_array_B_minus = invariant_masses_array_B_minus_no_resonances[background_lower_limit_minus_mask]

background_invariant_masses_array_B_minus = unbounded_background_invariant_masses_array_B_minus[:int(exp_count_plus)]


def find_upper_limit_mask (background_invariant_masses_array, exp_count):
    
    indices = np.argsort(background_invariant_masses_array)[:int(exp_count)]
    mask_indices = np.empty(int(exp_count), dtype=int)
    mask_indices[:] = indices
    
    return mask_indices

upper_limit_bg_array_plus = find_upper_limit_mask(unbounded_background_invariant_masses_array_B_plus,exp_count_plus)
upper_limit_B_plus_background = np.max(unbounded_background_invariant_masses_array_B_plus[upper_limit_bg_array_plus])



###

upper_limit_B_minus_background = np.max(background_invariant_masses_array_B_minus)

background_count_B_plus = len(background_invariant_masses_array_B_plus)

background_count_B_minus = len(background_invariant_masses_array_B_minus)

upper_limit_bg_array_minus = find_upper_limit_mask(unbounded_background_invariant_masses_array_B_minus,exp_count_minus)
upper_limit_B_minus_background = np.max(unbounded_background_invariant_masses_array_B_minus[upper_limit_bg_array_minus])



In [ ]:
background_plus_mask = (invariant_masses_array_B_pm_no_resonances>lower_limit_background) & (invariant_masses_array_B_pm_no_resonances<upper_limit_B_plus_background)

background_kaon_charges = kaon_charges_no_resonances[upper_limit_bg_array_plus]


background_plus_kaon_energies = kaon_energies_no_resonances[upper_limit_bg_array_plus][background_kaon_charges==1]

background_plus_pion_1_energies = pion_1_energies_no_resonances[upper_limit_bg_array_plus][background_kaon_charges==1]

background_plus_pion_2_energies = pion_2_energies_no_resonances[upper_limit_bg_array_plus][background_kaon_charges==1]

background_plus_kaon_momenta_x = kaon_momenta_x_no_resonances[upper_limit_bg_array_plus][background_kaon_charges==1]

background_plus_kaon_momenta_y = kaon_momenta_y_no_resonances[upper_limit_bg_array_plus][background_kaon_charges==1]

background_plus_kaon_momenta_z = kaon_momenta_z_no_resonances[upper_limit_bg_array_plus][background_kaon_charges==1]

background_plus_pion_momenta_1_x = pion_momenta_1_x_no_resonances[upper_limit_bg_array_plus][background_kaon_charges==1]

background_plus_pion_momenta_1_y = pion_momenta_1_y_no_resonances[upper_limit_bg_array_plus][background_kaon_charges==1]

background_plus_pion_momenta_1_z = pion_momenta_1_z_no_resonances[upper_limit_bg_array_plus][background_kaon_charges==1]

background_plus_pion_momenta_2_x = pion_momenta_2_x_no_resonances[upper_limit_bg_array_plus][background_kaon_charges==1]

background_plus_pion_momenta_2_y = pion_momenta_2_y_no_resonances[upper_limit_bg_array_plus][background_kaon_charges==1]

background_plus_pion_momenta_2_z = pion_momenta_2_z_no_resonances[upper_limit_bg_array_plus][background_kaon_charges==1]

background_plus_kaon_charges = kaon_charges_no_resonances[upper_limit_bg_array_plus]


In [ ]:
K_Pi_1_inv_mass_B_plus_bg = two_body_invariant_mass(background_plus_kaon_energies, background_plus_pion_1_energies,
                                          (background_plus_kaon_momenta_x + background_plus_pion_momenta_1_x),
                                          (background_plus_kaon_momenta_y + background_plus_pion_momenta_1_y),
                                          (background_plus_kaon_momenta_z + background_plus_pion_momenta_1_z))

K_Pi_2_inv_mass_B_plus_bg = two_body_invariant_mass(background_plus_kaon_energies, background_plus_pion_2_energies,
                                          (background_plus_kaon_momenta_x + background_plus_pion_momenta_2_x),
                                          (background_plus_kaon_momenta_y + background_plus_pion_momenta_2_y),
                                          (background_plus_kaon_momenta_z + background_plus_pion_momenta_2_z))

Pi_Pi_inv_mass_B_plus_bg = two_body_invariant_mass(background_plus_pion_1_energies, background_plus_pion_2_energies,
                                          (background_plus_pion_momenta_1_x + background_plus_pion_momenta_2_x),
                                          (background_plus_pion_momenta_1_y + background_plus_pion_momenta_2_y),
                                          (background_plus_pion_momenta_1_z + background_plus_pion_momenta_2_z))


# Calculate the squared invariant masses
m12_squared_B_plus_bg = 1e-6 * np.square(K_Pi_1_inv_mass_B_plus_bg) # k pi +
m13_squared_B_plus_bg = 1e-6 * np.square(K_Pi_2_inv_mass_B_plus_bg) # k pi -
m23_squared_B_plus_bg = 1e-6 * np.square(Pi_Pi_inv_mass_B_plus_bg) # pi + pi -

# Create the Dalitz plot as a heatmap
plt.figure(figsize=(3, 3), dpi=500)
hist3 = plt.hist2d(m23_squared_B_plus_bg, m13_squared_B_plus_bg, bins=num_bins_x, cmap='seismic')

plt.colorbar(hist1[3])

plt.xlabel('$m_{pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
plt.ylabel('$m_{K^+\pi^-}^2$ (GeV$^2$/c$^4$)')
plt.suptitle('Dalitz Plot of the $B^+$ Background Region ($m_{\pi^+,\pi^-}^2$ Against $m_{K^+,\pi^-}^2$)')
plt.tight_layout()
plt.grid(True)

plt.show()

In [ ]:
background_minus_mask = (invariant_masses_array_B_pm_no_resonances>lower_limit_background) & (invariant_masses_array_B_pm_no_resonances<upper_limit_B_minus_background)

background_kaon_charges = kaon_charges_no_resonances[upper_limit_bg_array_minus]


background_minus_kaon_energies = kaon_energies_no_resonances[upper_limit_bg_array_minus][background_kaon_charges==-1]

background_minus_pion_1_energies = pion_1_energies_no_resonances[upper_limit_bg_array_minus][background_kaon_charges==-1]

background_minus_pion_2_energies = pion_2_energies_no_resonances[upper_limit_bg_array_minus][background_kaon_charges==-1]

background_minus_kaon_momenta_x = kaon_momenta_x_no_resonances[upper_limit_bg_array_minus][background_kaon_charges==-1]

background_minus_kaon_momenta_y = kaon_momenta_y_no_resonances[upper_limit_bg_array_minus][background_kaon_charges==-1]

background_minus_kaon_momenta_z = kaon_momenta_z_no_resonances[upper_limit_bg_array_minus][background_kaon_charges==-1]

background_minus_pion_momenta_1_x = pion_momenta_1_x_no_resonances[upper_limit_bg_array_minus][background_kaon_charges==-1]

background_minus_pion_momenta_1_y = pion_momenta_1_y_no_resonances[upper_limit_bg_array_minus][background_kaon_charges==-1]

background_minus_pion_momenta_1_z = pion_momenta_1_z_no_resonances[upper_limit_bg_array_minus][background_kaon_charges==-1]

background_minus_pion_momenta_2_x = pion_momenta_2_x_no_resonances[upper_limit_bg_array_minus][background_kaon_charges==-1]

background_minus_pion_momenta_2_y = pion_momenta_2_y_no_resonances[upper_limit_bg_array_minus][background_kaon_charges==-1]

background_minus_pion_momenta_2_z = pion_momenta_2_z_no_resonances[upper_limit_bg_array_minus][background_kaon_charges==-1]

background_minus_kaon_charges = kaon_charges_no_resonances[upper_limit_bg_array_minus]

In [ ]:
K_Pi_1_inv_mass_B_minus_bg = two_body_invariant_mass(background_minus_kaon_energies, background_minus_pion_1_energies,
                                          (background_minus_kaon_momenta_x + background_minus_pion_momenta_1_x),
                                          (background_minus_kaon_momenta_y + background_minus_pion_momenta_1_y),
                                          (background_minus_kaon_momenta_z + background_minus_pion_momenta_1_z))

K_Pi_2_inv_mass_B_minus_bg = two_body_invariant_mass(background_minus_kaon_energies, background_minus_pion_2_energies,
                                          (background_minus_kaon_momenta_x + background_minus_pion_momenta_2_x),
                                          (background_minus_kaon_momenta_y + background_minus_pion_momenta_2_y),
                                          (background_minus_kaon_momenta_z + background_minus_pion_momenta_2_z))

Pi_Pi_inv_mass_B_minus_bg = two_body_invariant_mass(background_minus_pion_1_energies, background_minus_pion_2_energies,
                                          (background_minus_pion_momenta_1_x + background_minus_pion_momenta_2_x),
                                          (background_minus_pion_momenta_1_y + background_minus_pion_momenta_2_y),
                                          (background_minus_pion_momenta_1_z + background_minus_pion_momenta_2_z))


# Calculate the squared invariant masses
m12_squared_B_minus_bg = 1e-6 * np.square(K_Pi_1_inv_mass_B_minus_bg) # k pi +
m13_squared_B_minus_bg = 1e-6 * np.square(K_Pi_2_inv_mass_B_minus_bg) # k pi -
m23_squared_B_minus_bg = 1e-6 * np.square(Pi_Pi_inv_mass_B_minus_bg) # pi + pi -

# Create the Dalitz plot as a heatmap
plt.figure(figsize=(3, 3), dpi=500)
hist4 = plt.hist2d(m23_squared_B_minus_bg, m12_squared_B_minus_bg, bins=num_bins_x, cmap='seismic')


plt.colorbar(hist1[3])

plt.xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
plt.ylabel('$m_{K^-\pi^+}^2$ (GeV$^2$/c$^4$)')
plt.suptitle('Dalitz Plot of the $B^-$ Background Region ($m_{K^-\pi^-}^2$ Against $m_{\pi^+,\pi^-}^2$)')
plt.tight_layout()
plt.grid(True)
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(18, 5))
fig.subplots_adjust(wspace=0.3) # increase horizontal space between plots

ax[0].hist2d(m23_squared_B_plus, m13_squared_B_plus, bins=num_bins_x, cmap='seismic')
ax[1].hist2d(m23_squared_B_plus_bg, m13_squared_B_plus_bg, bins=num_bins_x, cmap='seismic')

h2d1 = hist1
h2d2 = hist3

#print(len(m12_squared_B_plus))

# first calculate the bin centres from the bin boundaries of the hist2d object
xcentres_plus = []
ycentres_plus = []
for a,b in zip(h2d1[1][0:-1],h2d1[1][1:]):
    for c,d in zip(h2d1[2][0:-1],h2d1[2][1:]):
        xcentres_plus.append( (a+b)/2 )
        ycentres_plus.append( (c+d)/2 )

print(len(xcentres_plus))

# now extract the weights, that is the bin contents
w1 = (np.array(h2d1[0])).flatten()
w2 = (np.array(h2d2[0])).flatten()
wsub = np.subtract(w1,w2) # subtract bin content of two histograms
wsub_plus = np.maximum(wsub, 0)  # force any negative values to be 0

# produce the new histogram
h2d3_B_plus = ax[2].hist2d(xcentres_plus,ycentres_plus,weights=wsub_plus, bins = [h2d1[1],h2d1[2]],cmap = 'seismic') # recycle the binning from above

# plot the lot

ax[0].set_title('Dalitz Plot of the $B^+$ Central Gaussian Region')
ax[1].set_title('Dalitz Plot of the $B^+$ Background Region')
ax[2].set_title('Dalitz Plot of the $B^+$ Signal Events')

ax[0].set_xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
ax[0].set_ylabel('$m_{K^+\pi^-}^2$ (GeV$^2$/c$^4$)')

ax[1].set_xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
ax[1].set_ylabel('$m_{K^+\pi^-}^2$ (GeV$^2$/c$^4$)')

ax[2].set_xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
ax[2].set_ylabel('$m_{K^+\pi^-}^2$ (GeV$^2$/c$^4$)')

fig.colorbar(h2d1[3],ax=ax[0]) # let's add the colour scale for histo 1
fig.colorbar(h2d2[3],ax=ax[1]) # let's add the colour scale for histo 2
fig.colorbar(h2d3_B_plus[3],ax=ax[2]) # let's add the colour scale for histo 3
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(18, 5))
fig.subplots_adjust(wspace=0.3) # increase horizontal space between plots

print(len(m12_squared_B_minus))
print(m12_squared_B_minus)

ax[0].hist2d(m23_squared_B_minus, m12_squared_B_minus, bins=num_bins_x, cmap='seismic')
ax[1].hist2d(m23_squared_B_minus_bg, m12_squared_B_minus_bg, bins=num_bins_x, cmap='seismic')

h2d1 = hist2
h2d2 = hist4

#h2d1 = ax[0].hist2d(m12_squared_B_plus, m13_squared_B_plus, bins=num_bins_x, cmap='seismic')
#h2d2 = ax[1].hist2d(m12_squared_B_plus_bg, m13_squared_B_plus_bg, bins=num_bins_x, cmap='seismic')

# first calculate the bin centres from the bin boundaries of the hist2d object
xcentres_minus = []
ycentres_minus = []
for a,b in zip(h2d1[1][0:-1],h2d1[1][1:]):
    for c,d in zip(h2d1[2][0:-1],h2d1[2][1:]):
        xcentres_minus.append( (a+b)/2 )
        ycentres_minus.append( (c+d)/2 )

# now extract the weights, that is the bin contents
w1 = (np.array(h2d1[0])).flatten()
w2 = (np.array(h2d2[0])).flatten()
wsub_minus = np.subtract(w1,w2) # subtract bin content of two histograms
wsub_minus = np.maximum(wsub_minus, 0)  # force any negative values to be 0

# produce the new histogram
h2d3_B_minus = ax[2].hist2d(xcentres_minus,ycentres_minus,weights=wsub_minus, bins = [h2d1[1],h2d1[2]],cmap = 'seismic') # recycle the binning from above

ax[0].set_title('Dalitz Plot of the $B^-$ Central Gaussian Region')
ax[1].set_title('Dalitz Plot of the $B^-$ Background Region')
ax[2].set_title('Dalitz Plot of the $B^-$ Signal Events')

ax[0].set_xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
ax[0].set_ylabel('$m_{K^-\pi^+}^2$ (GeV$^2$/c$^4$)')

ax[1].set_xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
ax[1].set_ylabel('$m_{K^-\pi^+}^2$ (GeV$^2$/c$^4$)')

ax[2].set_xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
ax[2].set_ylabel('$m_{K^-\pi^+}^2$ (GeV$^2$/c$^4$)')

# plot the lot
fig.colorbar(h2d1[3],ax=ax[0]) # let's add the colour scale for histo 1
fig.colorbar(h2d2[3],ax=ax[1]) # let's add the colour scale for histo 2
fig.colorbar(h2d3_B_minus[3],ax=ax[2]) # let's add the colour scale for histo 3
plt.show()

## Local CP Asymmetry

In [ ]:
num_bins_x= 20
num_bins_y= 20

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(18, 5))
fig.subplots_adjust(wspace=0.3) # increase horizontal space between plots
#x_bins = [10000,50000,100000,2500000,5000000,30000000]
#y_bins = [10000,50000,100000,2500000,5000000,30000000]

h2d2 = ax[0].hist2d(xcentres_plus,ycentres_plus,weights=wsub_plus, bins = num_bins_x,cmap = 'seismic' )
h2d1 = ax[1].hist2d(xcentres_minus,ycentres_minus,weights=wsub_minus, bins = num_bins_x,cmap = 'seismic') 

ax[0].set_title(r'B$^+$ Signal Only')
ax[1].set_title(r'B$^-$ Signal Only')
ax[2].set_title(r'Signal Asymmetry')

ax[0].set_xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
ax[0].set_ylabel('$m_{K^+\pi^-}^2$ (GeV$^2$/c$^4$)')

ax[1].set_xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
ax[1].set_ylabel('$m_{K^-\pi^+}^2$ (GeV$^2$/c$^4$)')

ax[2].set_xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
ax[2].set_ylabel('$m_{K^±\pi^\mp}^2$ (GeV$^2$/c$^4$)')

#h2d1 = h2d3_B_minus
#h2d2 = h2d3_B_plus

# first calculate the bin centres from the bin boundaries of the hist2d object
xcentres_CP = []
ycentres_CP = []
for a,b in zip(h2d1[1][0:-1],h2d1[1][1:]):
    for c,d in zip(h2d1[2][0:-1],h2d1[2][1:]):
        xcentres_CP.append( (a+b)/2 )
        ycentres_CP.append( (c+d)/2 )

# now extract the weights, that is the bin contents
w1 = (np.array(h2d1[0])).flatten()
w2 = (np.array(h2d2[0])).flatten()

wAsymm_CP = np.zeros_like(w1)  # Initialize wAsymm_CP with zeros, same shape as w1

for i in range(len(w1)):
    if w1[i] + w2[i] == 0:
        wAsymm_CP[i] = 0
    else:
        wAsymm_CP[i] = (w1[i] - w2[i]) / (w1[i] + w2[i])
        

x_bins = [0,2,5,7.5,10,15,17.5,20,22.5,25,26,28,30]
y_bins = [0,3,5,7.5,10,15,17.5,20,22.5,25,26,28,30]

data_min = np.min(wAsymm_CP)
data_max = np.max(wAsymm_CP)
limit = max(abs(data_min), abs(data_max))

# Creating the Normalize object
norm = Normalize(vmin=-limit, vmax=limit)

# produce the new histogram
h2d3_local_asymmetry = ax[2].hist2d(xcentres_CP,ycentres_CP,weights=wAsymm_CP, bins = [x_bins,y_bins],cmap = 'seismic',norm=norm)
# plot the lot
fig.colorbar(h2d1[3],ax=ax[0]) # let's add the colour scale for histo 1
fig.colorbar(h2d2[3],ax=ax[1]) # let's add the colour scale for histo 2
fig.colorbar(h2d3_local_asymmetry[3],ax=ax[2]) # let's add the colour scale for histo 3
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(18, 5))
fig.subplots_adjust(wspace=0.3) # increase horizontal space between plots

h2d2 = ax[0].hist2d(xcentres_plus,ycentres_plus,weights=wsub_plus, bins = num_bins_x,cmap = 'seismic' )
h2d1 = ax[1].hist2d(xcentres_minus,ycentres_minus,weights=wsub_minus, bins = num_bins_x,cmap = 'seismic') 
ax[0].set_title(r'B$^+$ signal only')
ax[1].set_title(r'B$^-$ signal only')
ax[2].set_title('Three Sigma on Asymmetry')

#h2d1 = h2d3_B_minus
#h2d2 = h2d3_B_plus

# first calculate the bin centres from the bin boundaries of the hist2d object
xcentres_CP_error = []
ycentres_CP_error = []
for a,b in zip(h2d1[1][0:-1],h2d1[1][1:]):
    for c,d in zip(h2d1[2][0:-1],h2d1[2][1:]):
        xcentres_CP_error.append( (a+b)/2 )
        ycentres_CP_error.append( (c+d)/2 )

# now extract the weights, that is the bin contents
w1 = (np.array(h2d1[0])).flatten()
w2 = (np.array(h2d2[0])).flatten()

werr_CP = np.zeros_like(w1)  # Initialize wAsymm_CP with zeros, same shape as w1

for i in range(len(w1)):
    if w1[i] + w2[i] == 0:
        werr_CP[i] = 0
    else:
        werr_CP[i] = np.multiply(3,np.sqrt((1-np.square(np.divide((w1[i] - w2[i]),(w1[i] + w2[i])))) / (w1[i] + w2[i])))

ax[0].set_xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
ax[0].set_ylabel('$m_{K^+\pi^-}^2$ (GeV$^2$/c$^4$)')

ax[1].set_xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
ax[1].set_ylabel('$m_{K^-\pi^+}^2$ (GeV$^2$/c$^4$)')

ax[2].set_xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
ax[2].set_ylabel('$m_{K^±\pi^\mp}^2$ (GeV$^2$/c$^4$)')


# produce the new histogram
h2d3_local_asymmetry = ax[2].hist2d(xcentres_CP_error,ycentres_CP_error,weights=werr_CP, bins = [h2d1[1],h2d1[2]],cmap = 'seismic')
# plot the lot
fig.colorbar(h2d1[3],ax=ax[0]) # let's add the colour scale for histo 1
fig.colorbar(h2d2[3],ax=ax[1]) # let's add the colour scale for histo 2
fig.colorbar(h2d3_local_asymmetry[3],ax=ax[2]) # let's add the colour scale for histo 3
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(18, 5))
fig.subplots_adjust(wspace=0.3) # increase horizontal space between plots

h2d2 = ax[0].hist2d(xcentres_plus,ycentres_plus,weights=wsub_plus, bins = num_bins_x,cmap = 'seismic' )
h2d1 = ax[1].hist2d(xcentres_minus,ycentres_minus,weights=wsub_minus, bins = num_bins_x,cmap = 'seismic') 
ax[0].set_title(r'B$^+$ signal only')
ax[1].set_title(r'B$^-$ signal only')
ax[2].set_title('CP Violation Magnitude')
ax[2].text(0.5, 0.95, 'Greater than three sigma', ha='center', va='top', transform=ax[2].transAxes, fontsize='smaller', color='white')
#h2d1 = h2d3_B_minus
#h2d2 = h2d3_B_plus

ax[0].set_xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
ax[0].set_ylabel('$m_{K^+\pi^-}^2$ (GeV$^2$/c$^4$)')

ax[1].set_xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
ax[1].set_ylabel('$m_{K^-\pi^+}^2$ (GeV$^2$/c$^4$)')

ax[2].set_xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
ax[2].set_ylabel('$m_{K^±\pi^\mp}^2$ (GeV$^2$/c$^4$)')

# first calculate the bin centres from the bin boundaries of the hist2d object
xcentres_CP = []
ycentres_CP = []
for a,b in zip(h2d1[1][0:-1],h2d1[1][1:]):
    for c,d in zip(h2d1[2][0:-1],h2d1[2][1:]):
        xcentres_CP.append( (a+b)/2 )
        ycentres_CP.append( (c+d)/2 )

# now extract the weights, that is the bin contents
w1 = (np.array(h2d1[0])).flatten()
w2 = (np.array(h2d2[0])).flatten()

evidence_CP = np.zeros_like(w1)  # Initialize evidence_CP with zeros, same shape as w1

for i in range(len(w1)):
    if np.abs(wAsymm_CP[i]) > werr_CP[i]:
        evidence_CP[i] = np.abs(wAsymm_CP[i])
    else:
        evidence_CP[i] = 0

data_min = np.min(evidence_CP)
data_max = np.max(evidence_CP)
limit = max(abs(data_min), abs(data_max))

# Creating the Normalize object
norm = Normalize(vmin=-limit, vmax=limit)

# produce the new histogram
h2d3_local_asymmetry = ax[2].hist2d(xcentres_CP,ycentres_CP,weights=evidence_CP, bins = [h2d1[1],h2d1[2]],cmap = 'seismic', norm=norm)
# plot the lot
fig.colorbar(h2d1[3],ax=ax[0]) # let's add the colour scale for histo 1
fig.colorbar(h2d2[3],ax=ax[1]) # let's add the colour scale for histo 2
fig.colorbar(h2d3_local_asymmetry[3],ax=ax[2]) # let's add the colour scale for histo 3
plt.show()

In [ ]:
M_B = 5.279  # B meson mass in GeV/c^2
m_pi = 0.140  # Charged pion mass in GeV/c^2
m_K = 0.494  # Charged kaon mass in GeV/c^2

def λ (x,y,z):
    return np.square(x) + np.square(y) + np.square(z) - 2*x*y - 2*y*z - 2*z*x

# Function to calculate the boundary of the Dalitz plot
'''def dalitz_boundary(m_pi_plus_pi_minus_squared):
    # Only calculate the boundary where it's physically meaningful
    if m_pi_plus_pi_minus_squared <= (M_B - m_K - m_pi)**2 and m_pi_plus_pi_minus_squared >= (m_K + m_pi)**2:
        return m_pi**2 + m_K**2 + (1 / (2 * m_pi_plus_pi_minus_squared)) * \
               ((M_B**2 - m_pi_plus_pi_minus_squared - m_pi**2) *
                (m_pi_plus_pi_minus_squared - m_K**2 + m_pi**2)**0.5 *
                (m_pi_plus_pi_minus_squared - m_K**2 - m_pi**2)**0.5)
    else:
        return np.nan'''

def dalitz_boundary_new(m_pi_plus_pi_minus_squared):
        return m_pi**2 + m_K**2 + (1 / (2 * m_pi_plus_pi_minus_squared)) * \
               ((M_B**2 - m_pi_plus_pi_minus_squared - m_K**2) *
                (m_pi_plus_pi_minus_squared - m_pi**2 + m_pi**2) +
                np.sqrt(λ(m_pi_plus_pi_minus_squared, M_B**2, m_K**2))*np.sqrt(λ(m_pi_plus_pi_minus_squared, m_pi**2, m_pi**2)))

def dalitz_boundary_new_lower(m_pi_plus_pi_minus_squared):
        return m_pi**2 + m_K**2 + (1 / (2 * m_pi_plus_pi_minus_squared)) * \
               ((M_B**2 - m_pi_plus_pi_minus_squared - m_K**2) *
                (m_pi_plus_pi_minus_squared - m_pi**2 + m_pi**2) -
                np.sqrt(λ(m_pi_plus_pi_minus_squared, M_B**2, m_K**2))*np.sqrt(λ(m_pi_plus_pi_minus_squared, m_pi**2, m_pi**2)))
    
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(18, 5))
fig.subplots_adjust(wspace=0.3) # increase horizontal space between plots

h2d2 = ax[0].hist2d(xcentres_plus,ycentres_plus,weights=wsub_plus, bins = num_bins_x,cmap = 'seismic' )
h2d1 = ax[1].hist2d(xcentres_minus,ycentres_minus,weights=wsub_minus, bins = num_bins_x,cmap = 'seismic') 
ax[0].set_title(r'B$^+$ signal only')
ax[1].set_title(r'B$^-$ signal only')
ax[2].set_title('Significance of CP Violation')
#h2d1 = h2d3_B_minus
#h2d2 = h2d3_B_plus

ax[0].set_xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
ax[0].set_ylabel('$m_{K^+\pi^-}^2$ (GeV$^2$/c$^4$)')

ax[1].set_xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
ax[1].set_ylabel('$m_{K^-\pi^+}^2$ (GeV$^2$/c$^4$)')

ax[2].set_xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
ax[2].set_ylabel('$m_{K^±\pi^\mp}^2$ (GeV$^2$/c$^4$)')

# first calculate the bin centres from the bin boundaries of the hist2d object
xcentres_CP = []
ycentres_CP = []
for a,b in zip(h2d1[1][0:-1],h2d1[1][1:]):
    for c,d in zip(h2d1[2][0:-1],h2d1[2][1:]):
        xcentres_CP.append( (a+b)/2 )
        ycentres_CP.append( (c+d)/2 )

# now extract the weights, that is the bin contents
w1 = (np.array(h2d1[0])).flatten()
w2 = (np.array(h2d2[0])).flatten()

significance_CP = np.zeros_like(w1)  # Initialize evidence_CP with zeros, same shape as w1

for i in range(len(w1)):
    if w1[i] + w2[i] == 0:
        wAsymm_CP[i] = 0
        significance_CP[i] = 0
    elif np.multiply(1,np.sqrt((1-np.square(np.divide((w1[i] - w2[i]),(w1[i] + w2[i])))) / (w1[i] + w2[i]))) == 0:
        wAsymm_CP[i] = 0
        significance_CP[i] = 0
    else:
        wAsymm_CP[i] = (w1[i] - w2[i]) / (w1[i] + w2[i])
        werr_CP[i] = np.multiply(1,np.sqrt((1-np.square(np.divide((w1[i] - w2[i]),(w1[i] + w2[i])))) / (w1[i] + w2[i])))
        significance_CP[i] = wAsymm_CP[i]/werr_CP[i]

data_min = np.min(significance_CP)
data_max = np.max(significance_CP)
limit = max(abs(data_min), abs(data_max))

# Creating the Normalize object
norm = Normalize(vmin=-limit, vmax=limit)

xcentres_CP = np.array(xcentres_CP)
ycentres_CP = np.array(ycentres_CP)

# Calculate the boundary for each x centre
#boundary_y = np.array([dalitz_boundary(x) for x in xcentres_plus])
boundary_values = [dalitz_boundary_new(x) for x in xcentres_CP]
boundary_values_lower = [dalitz_boundary_new_lower(x) for x in xcentres_CP]

for i, x_center in enumerate(xcentres_CP):
    for j, y_center in enumerate(ycentres_CP):
        # Check if the y center is above the boundary for the corresponding x center
        if y_center > boundary_values[i] or y_center < boundary_values_lower[i]:
            # Calculate the index for the flattened array
            index = i * len(ycentres_CP) + j
            # Check if the index is within the bounds of the flattened significance array
            if index < len(significance_CP):
                significance_CP[index] = np.nan

# produce the new histogram
h2d3_local_asymmetry = ax[2].hist2d(xcentres_CP,ycentres_CP,weights=significance_CP, bins = num_bins_x,cmap = 'seismic',norm=norm)
# plot the lot
fig.colorbar(h2d1[3],ax=ax[0]) # let's add the colour scale for histo 1
fig.colorbar(h2d2[3],ax=ax[1]) # let's add the colour scale for histo 2
fig.colorbar(h2d3_local_asymmetry[3],ax=ax[2]) # let's add the colour scale for histo 3
plt.show()

In [ ]:
import matplotlib.patches as mpatches

# Assuming M_B, m_pi, m_K, dalitz_boundary_new, dalitz_boundary_new_lower are defined

# Create a grid of x and y values
x_values = np.linspace(-1, (M_B - m_K - m_pi)**2 + 2, 1000)
y_values = np.linspace(-1, M_B**2, 1000)

# Initialize a 2D histogram array
mock_histogram = np.zeros((len(x_values), len(y_values)))

# Calculate the boundary and fill the histogram array
for i, x in enumerate(x_values):
    boundary = dalitz_boundary_new(x)
    boundary_lower = dalitz_boundary_new_lower(x)
    for j, y in enumerate(y_values):
        if y > boundary or y < boundary_lower or np.isnan(boundary):  # Check if the bin is above or below the boundary or if boundary is NaN
            mock_histogram[i, j] = 1  # Mark as not-allowed

# Plot the mock histogram
plt.figure(figsize=(8, 6))
cmap = plt.get_cmap('winter')
plt.pcolormesh(x_values, y_values, mock_histogram.T, cmap=cmap, shading='auto')

allowed_color = cmap(0.01)  # A color closer to one end of the colormap
not_allowed_color = cmap(0.99)  # A color closer to the other end of the colormap
# Create custom patches for the legend
allowed_patch = mpatches.Patch(color=allowed_color, label='Allowed')
not_allowed_patch = mpatches.Patch(color=not_allowed_color, label='Not Allowed')

# Add legend
plt.legend(handles=[allowed_patch, not_allowed_patch])

plt.xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
plt.ylabel('$m_{K^\pm\pi^\mp}^2$ (GeV$^2$/c$^4$)')
plt.title('Theoretical Dalitz Plot')
plt.show()


In [ ]:
import matplotlib.patches as mpatches
# Create a grid of x and y values
#x_values = np.linspace((m_K + m_pi)**2, (M_B - m_K - m_pi)**2, 100)
x_values = np.linspace(-1, (M_B - m_K - m_pi)**2 + 2, 100)
y_values = np.linspace(-1, M_B**2, 100)

# Initialize a 2D histogram array
mock_histogram = np.zeros((len(x_values), len(y_values)))

# Calculate the boundary and fill the histogram array
for i, x in enumerate(x_values):
    boundary = dalitz_boundary_new(x)
    boundary_lower = dalitz_boundary_new_lower(x)
    for j, y in enumerate(y_values):
        if y > boundary or y < boundary_lower or np.isnan(boundary):  # Check if the bin is above the boundary or if boundary is NaN
            mock_histogram[i, j] = 1  # Mark as not-allowed

# Plot the mock histogram
plt.figure(figsize=(8, 6))
plt.pcolormesh(x_values, y_values, mock_histogram.T, cmap='RdBu', shading='auto')
plt.xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
plt.ylabel('$m_{K^\pm\pi^\mp}^2$ (GeV$^2$/c$^4$)')
plt.title('Mock Dalitz Plot Boundary')
plt.show()

In [ ]:
hist, bin_edges = np.histogram(significance_CP, bins=15)
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

# Creating a scatter plot


# Calculate errors (assuming Poisson statistics, so error is sqrt of count)
errors = np.sqrt(hist)

# Creating a scatter plot using bin centers and histogram values with error bars

def gaussian_local(x, amplitude, standard_deviation):
    return amplitude * np.exp( - ((x - 0)**2 / (2*standard_deviation**2)))

popt, _ = curve_fit(gaussian_local, bin_centers, hist, maxfev=100000, p0=[max(hist), 1])

x_interval_for_fit = np.linspace(bin_edges[0], bin_edges[-1], 1000)


"""plt.xlabel('Bin Centers')
plt.ylabel('Frequency')
plt.title('Scatter Plot of Bin Centers vs Frequency')
matplotlib.figsize(100)
plt.show()"""

fig,ax = plt.subplots(nrows=1,ncols=1,figsize = (10,6))
ax.set_xlabel('Significance of CP Violation')
ax.set_ylabel('Entries')
ax.errorbar(bin_centers, hist, yerr=errors, fmt='o')
ax.plot(x_interval_for_fit, gaussian_local(x_interval_for_fit, *popt), label='Fitted function')

plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(18, 5))
fig.subplots_adjust(wspace=0.3) # increase horizontal space between plots

h2d2 = ax[0].hist2d(xcentres_plus,ycentres_plus,weights=wsub_plus, bins = num_bins_x,cmap = 'seismic' )
h2d1 = ax[1].hist2d(xcentres_minus,ycentres_minus,weights=wsub_minus, bins = num_bins_x,cmap = 'seismic') 
ax[0].set_title(r'B$^+$ signal only')
ax[1].set_title(r'B$^-$ signal only')
ax[2].set_title('Significance of CP Violation')
#h2d1 = h2d3_B_minus
#h2d2 = h2d3_B_plus

ax[0].set_xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
ax[0].set_ylabel('$m_{K^+\pi^-}^2$ (GeV$^2$/c$^4$)')

ax[1].set_xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
ax[1].set_ylabel('$m_{K^-\pi^+}^2$ (GeV$^2$/c$^4$)')

ax[2].set_xlabel('$m_{\pi^+,\pi^-}^2$ (GeV$^2$/c$^4$)')
ax[2].set_ylabel('$m_{K^±\pi^\mp}^2$ (GeV$^2$/c$^4$)')

# first calculate the bin centres from the bin boundaries of the hist2d object
xcentres_CP = []
ycentres_CP = []
for a,b in zip(h2d1[1][0:-1],h2d1[1][1:]):
    for c,d in zip(h2d1[2][0:-1],h2d1[2][1:]):
        xcentres_CP.append( (a+b)/2 )
        ycentres_CP.append( (c+d)/2 )

# now extract the weights, that is the bin contents
w1 = (np.array(h2d1[0])).flatten()
w2 = (np.array(h2d2[0])).flatten()

significance_CP = np.zeros_like(w1)  # Initialize evidence_CP with zeros, same shape as w1

for i in range(len(w1)):
    if w1[i] + w2[i] == 0:
        wAsymm_CP[i] = 0
        significance_CP[i] = 0
    elif np.multiply(1,np.sqrt((1-np.square(np.divide((w1[i] - w2[i]),(w1[i] + w2[i])))) / (w1[i] + w2[i]))) == 0:
        wAsymm_CP[i] = 0
        significance_CP[i] = 0
    else:
        wAsymm_CP[i] = (w1[i] - w2[i]) / (w1[i] + w2[i])
        werr_CP[i] = np.multiply(1,np.sqrt((1-np.square(np.divide((w1[i] - w2[i]),(w1[i] + w2[i])))) / (w1[i] + w2[i])))
        significance_CP[i] = wAsymm_CP[i]/werr_CP[i]


print(len(xcentres_plus))

data_min = np.min(significance_CP)
data_max = np.max(significance_CP)
limit = max(abs(data_min), abs(data_max))

# Creating the Normalize object
norm = Normalize(vmin=-limit, vmax=limit)
        
x_bins = [0,2,5,7.5,10,12.5,15,17.5,20,22.5,25,26,28,30]
y_bins = [0,1.5,3,5,7.5,10,15,17, 18.5,20,22.5,25,26,28,30]

boundary_values = [dalitz_boundary_new(x) for x in xcentres_CP]

'''for i, x_center in enumerate(xcentres_CP):
    for j, y_center in enumerate(ycentres_CP):
        # Check if the y center is above the boundary for the corresponding x center
        if y_center > boundary_values[i]:
            # Calculate the index for the flattened array
            index = i * len(ycentres_CP) + j
            # Check if the index is within the bounds of the flattened significance array
            if index < len(significance_CP):
                significance_CP[index] = 0'''


# produce the new histogram
#h2d3_local_asymmetry = ax[2].hist2d(xcentres_CP,ycentres_CP,weights=significance_CP, bins = [x_bins,y_bins],cmap = 'seismic')
h2d3_local_asymmetry = ax[2].hist2d(xcentres_CP, ycentres_CP, weights=significance_CP, bins=[x_bins, y_bins], cmap='seismic', norm=norm)

#Â plot the lot
fig.colorbar(h2d1[3],ax=ax[0]) # let's add the colour scale for histo 1
fig.colorbar(h2d2[3],ax=ax[1]) # let's add the colour scale for histo 2
fig.colorbar(h2d3_local_asymmetry[3],ax=ax[2]) # let's add the colour scale for histo 3
plt.show()

In [ ]:
print(len(m12_squared_B_minus))
print(m12_squared_B_minus)
print(len(m13_squared_B_minus))

# Applying the condition
condition = ((m23_squared_B_minus > 0) & (m23_squared_B_minus < 2)) & ((m13_squared_B_minus > 10) & (m13_squared_B_minus < 15))

# Using the condition to filter xcentres_CP
m23_squared_B_minus_region = m23_squared_B_minus[condition]
m13_squared_B_minus_region = m13_squared_B_minus[condition]
print(len(m23_squared_B_minus_region))
print(len(m13_squared_B_minus_region))

### Local Asymmetry Analysis (in more detail)

In [ ]:
K_Pi_plus_inv_mass = two_body_invariant_mass(kaon_energies_no_resonances, pion_1_energies_no_resonances,
                                          (kaon_momenta_x_no_resonances + pion_momenta_1_x_no_resonances),
                                          (kaon_momenta_y_no_resonances + pion_momenta_1_y_no_resonances),
                                          (kaon_momenta_z_no_resonances + pion_momenta_1_z_no_resonances))

K_Pi_minus_inv_mass = two_body_invariant_mass(kaon_energies_no_resonances, pion_2_energies_no_resonances,
                                          (kaon_momenta_x_no_resonances + pion_momenta_2_x_no_resonances),
                                          (kaon_momenta_y_no_resonances + pion_momenta_2_y_no_resonances),
                                          (kaon_momenta_z_no_resonances + pion_momenta_2_z_no_resonances))

Pi_Pi_inv_mass = two_body_invariant_mass(pion_1_energies_no_resonances, pion_2_energies_no_resonances,
                                        (pion_momenta_1_x_no_resonances + pion_momenta_2_x_no_resonances),
                                        (pion_momenta_1_y_no_resonances + pion_momenta_2_y_no_resonances),
                                        (pion_momenta_1_z_no_resonances + pion_momenta_2_z_no_resonances))

local_asym_mask = ((np.square(Pi_Pi_inv_mass) > 0) & (np.square(Pi_Pi_inv_mass) < 2e6)) & ((np.square(K_Pi_minus_inv_mass) > 10e6) & (np.square(K_Pi_minus_inv_mass) < 15e6))
#print(len(K_Pi_minus_inv_mass))
#print(len(Pi_Pi_inv_mass))

Pi_Pi_inv_mass_filtered = Pi_Pi_inv_mass[local_asym_mask]

#local_asym_mask_k_pi_minus = (K_Pi_minus_inv_mass>0) & (K_Pi_minus_inv_mass<0.3e7)
K_Pi_minus_inv_mass_filtered = K_Pi_minus_inv_mass[local_asym_mask]

invariant_masses_array_B_pm_filtered = invariant_masses_array_B_pm_no_resonances[local_asym_mask]
kaon_charges_filtered = kaon_charges_no_resonances[local_asym_mask]

In [ ]:
print(len(K_Pi_minus_inv_mass))
print(len(K_Pi_minus_inv_mass_filtered))

print(len(K_Pi_minus_inv_mass_filtered)/len(K_Pi_minus_inv_mass))

In [ ]:

invariant_masses_array_B_plus_filtered = invariant_masses_array_B_pm_filtered[kaon_charges_filtered == 1]


invariant_masses_array_B_minus_filtered = invariant_masses_array_B_pm_filtered[kaon_charges_filtered == -1]

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 6))
#fig.subplots_adjust(wspace=0.3) # increase horizontal space between plots

# This plots two 1D-histograms.
# The color is changed automatically, the styles are set by hand
# keep hold of the pT histogram data for fitting later

num_bins = 200

data_B_plus_hist = ax.hist(invariant_masses_array_B_plus_filtered, bins = num_bins, range = [4000,6500],histtype = 'step',
                          color = 'c',label = '$B^{+}$')
data_B_minus_hist = ax.hist(invariant_masses_array_B_minus_filtered, bins = num_bins, range = [4000,6500],histtype = 'step',
                           color = 'black',label = '$B^{-}$')
ax.axvline(5280, color='green', linestyle='-',label = r'$M_{B^+}$')
bin_width = round((6500-4000)/num_bins)
ax.legend()
ax.grid(True,ls=':')
ax.set_xlabel('Invariant Mass (MeV)')
ax.set_ylabel(f'Entries per {bin_width} MeV')
ax.set_title('Invariant Mass Distribution (Local CP Asymmetry Analysis) of $B^+$ and $B^-$ Mesons')
ax.legend()
plt.show()

### Eazo's attempts to fit the local distributions start here

In [ ]:
num_bins = 100

data_B_plus_hist_local = ax.hist(invariant_masses_array_B_plus_filtered, bins = num_bins, range = [5200,5600],histtype = 'step',
                          color = 'c',label = '$B^{+}$')
data_B_minus_hist_local = ax.hist(invariant_masses_array_B_minus_filtered, bins = num_bins, range = [5200,5600],histtype = 'step',
                           color = 'black',label = '$B^{-}$')


y_b_minus_local = data_B_minus_hist_local[0]
x_b_minus_local = data_B_minus_hist_local[1]
x_b_minus_local = np.delete(x_b_minus_local,-1)

y_b_plus_local = data_B_plus_hist_local[0]
x_b_plus_local = data_B_plus_hist_local[1]
x_b_plus_local = np.delete(x_b_plus_local,-1)

#y_b_pm_local = data_B_pm_hist_local[0]
#x_b_pm_local = data_B_pm_hist_local[1]
#x_b_pm_local = np.delete(x_b_pm_local,-1)

In [ ]:
start_val_local = 5200
fixed_gaussian_peak_local = 5200
max_val_local = 5600

In [ ]:
#mask_fit_local = (x_b_pm>start_val_local) & (x_b_pm<max_val_local)
#x_b_pm_new_local = x_b_pm_local[mask_fit_local]
#y_b_pm_new_local = y_b_pm_local[mask_fit_local]
#y_b_pm_new_local = np.delete(y_b_pm_new_local,np.argmin(x_b_pm_new_local))

In [ ]:
mask_fit_local = (x_b_minus_local>start_val_local) & (x_b_minus_local<max_val_local)
x_b_minus_new_local = x_b_minus_local[mask_fit_local]
y_b_minus_new_local = y_b_minus_local[mask_fit_local]
y_b_minus_new_local = np.delete(y_b_minus_new_local,np.argmin(x_b_minus_new_local))

In [ ]:
mask_fit_local = (x_b_plus_local>start_val_local) & (x_b_plus_local<max_val_local)
x_b_plus_new_local = x_b_plus_local[mask_fit_local]
y_b_plus_new_local = y_b_plus_local[mask_fit_local]
y_b_plus_new_local = np.delete(y_b_plus_new_local,np.argmin(x_b_plus_new_local))

In [ ]:
def plot_results_local(ax,bin_centres,bin_centres_red,values,coeff_fit,fname, plus_or_minus):
  # plot the data, this time as dots with error bars (sqrt(N) errors)
    
    ax.errorbar(bin_centres_red,values,yerr=np.sqrt(values),linestyle='',marker='.',markerfacecolor='k',markeredgecolor='k',ecolor='k',label='Data')
    fit_x_vals = np.linspace(bin_centres_red[0],bin_centres_red[-1],1000)
  # plot the fit: create x values, then calculate the corresponding y values and plot
    y_fit = [fit_function(x,coeff_fit[0],coeff_fit[1],coeff_fit[2]
                             ,coeff_fit[3]) for x in fit_x_vals]
    ax.plot(fit_x_vals,y_fit,label='Fit',color='r',zorder=10) # zorder makes sure the fit line is on top
    ax.plot(fit_x_vals,exponential(fit_x_vals,coeff_fit[0],coeff_fit[1]),color = 'g',label = 'Exponential Function')
    ax.plot(fit_x_vals, crystal_ball_scipy(fit_x_vals,coeff_fit[2],coeff_fit[3]), color='c', label='Crystal Ball Function')
    ax.plot(fit_x_vals, gaussian(fit_x_vals, fixed_gaussian_amp, fixed_gaussian_peak, fixed_gaussian_std), color='y', label='Gaussian Function')

    ax.legend()
    ax.set_xlabel('$m_{inv}$ in MeV')
    ax.set_ylabel(f'Candidates per {bin_width} MeV')
    ax.set_title(f'(Local) Invariant Mass Distribution Fit $B^{plus_or_minus}$')
    #plt.savefig(fname)
    
    return np.array([coeff_fit[2],coeff_fit[3]]),np.array([coeff_fit[0],coeff_fit[1]])

In [ ]:
p0 = [3250, 1500, 5280, 100000]

fixed_gaussian_peak = start_val_local
fixed_gaussian_std = 20
fixed_gaussian_amp = 450
#normE,decay, m, meanCB, sigmaCB, normCB

sigma = np.sqrt(np.array(y_b_plus_new_local))
coeff_pT,cov_pT, bin_centres_pT_plus, bin_centres_red_pT, chi2_pT, ndf_pT = fit_data(x_b_plus_new_local, y_b_plus_new_local, start_val_local, max_val_local, p0, sigma)
                                                        #normE, decay, amplitude1,mean1, stddev1


#print(x_new[y.argmax()])
print_results(coeff_pT,cov_pT, chi2_pT, ndf_pT)
#
# plot results
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 6), dpi=1000)
gaussian_data_plus,exponential_data_plus = plot_results_local(ax,bin_centres_pT_plus,bin_centres_red_pT,y_b_plus_new_local,coeff_pT,'fit_pT.pdf', '+')


In [ ]:
p0 = [3250, 1500, 5280, 100000]

fixed_gaussian_peak = start_val_local
fixed_gaussian_std = 20
fixed_gaussian_amp = 500
#normE,decay, m, meanCB, sigmaCB, normCB

sigma = np.sqrt(np.array(y_b_minus_new_local))
coeff_pT,cov_pT, bin_centres_pT_minus, bin_centres_red_pT, chi2_pT, ndf_pT = fit_data(x_b_minus_new_local, y_b_minus_new_local, start_val_local, max_val_local, p0, sigma)
                                                        #normE, decay, amplitude1,mean1, stddev1


#print(x_new[y.argmax()])
print_results(coeff_pT,cov_pT, chi2_pT, ndf_pT)
#
# plot results
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 6), dpi=1000)
gaussian_data_minus,exponential_data_minus = plot_results_local(ax,bin_centres_pT_minus,bin_centres_red_pT,y_b_minus_new_local,coeff_pT,'fit_pT.pdf', '-')


In [ ]:
N_minus, N_minus_error = quad(crystal_ball_scipy, 0, np.inf, args=(gaussian_data_minus[0], gaussian_data_minus[1]))
N_plus, N_plus_error = quad(crystal_ball_scipy, 0, np.inf, args=(gaussian_data_plus[0], gaussian_data_plus[1]))

print("N_minus:", N_minus,'±',N_minus_error)
print("N_plus:", N_plus,'±',N_plus_error)

# The CP Asymmetry is
A = (N_minus - N_plus)/(N_minus + N_plus)
error_A = np.sqrt((1-A**2)/(N_plus+N_minus))

print("A:", A,'±',error_A)